In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# BEGIN NEW

### OLD FUNCTIONS TO BE UPDATED

In [ ]:
# MOVED TO OutageMdlrPrep
def get_active_SNs_for_xfmrs_OLD(
    trsf_pole_nbs, 
    df_mp_curr, 
    df_mp_hist,
    no_outg_time_infos_df=None, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    files_dir_no_outg=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\EndEvents_NoOutg', 
    file_path_glob_no_outg = r'end_events_[0-9]*.csv', 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    
    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    If no_outg_time_infos_df is not supplied, it will be built.
      files_dir_no_outg and file_path_glob_no_outg are only used when no_outg_time_infos_df needs built
    """
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Instead of a DF with trsf_pole_nb index and prem_nb column, we want opposite
    xfmr_for_PNs_df = PNs_for_xfmrs.explode(return_prem_nbs_col)
    xfmr_for_PNs_df[xfmr_for_PNs_df.index.name] = xfmr_for_PNs_df.index
    xfmr_for_PNs_df = xfmr_for_PNs_df.set_index(return_prem_nbs_col)  
    #-------------------------
    # If no_outg_time_infos_df is None, build it.  
    #   no_outg_time_infos_df has prem_nbs indices and t_min, t_max columns
    #   This is where the time information for each premise number comes from
    if no_outg_time_infos_df is None:
        paths_no_outg = Utilities.find_all_paths(base_dir=files_dir_no_outg, glob_pattern=file_path_glob_no_outg)
        no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
            summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths_no_outg], 
            output_prem_nbs_col=return_prem_nbs_col, 
            output_t_min_col=t_min_col, 
            output_t_max_col=t_max_col, 
            make_prem_nbs_idx=True, 
            include_summary_paths=False
        )    
    #-------------------------
    # Merge xfmr_for_PNs_df with no_outg_time_infos_df to append the time data to the former
    # NOTE: It is possible for t_min/t_max to be NaT (NaN) for some entries after the merge, meaning that the 
    #       premise numbers were not found in no_outg_time_infos_df
    #       This happens because these premise numbers must not have had any meter events, and thus were not included 
    #         in the SQL query (as it takes a long time to find empty results, so I weed these out before running the 
    #         query), and therefore the premise numbers were not found in the summary files/no_outg_time_infos_df.    
    xfmr_for_PNs_df = pd.merge(xfmr_for_PNs_df, no_outg_time_infos_df, how='left', left_index=True, right_index=True)
    #-------------------------
    # Want to consolidate xfmr_for_PNs_df, grouping by trsf_pole_nb and collecting t_min,t_max, and a list
    # of the premise numbers.  Therefore, first the index must be reset to make a PNs columns
    xfmr_for_PNs_df=xfmr_for_PNs_df.reset_index()
    #-----
    # Consolidate xfmr_for_PNs_df
    # NOTE: If t_min/t_max is NaT (NaN) for all premise numbers in a given trsf_pole_nb (see NOTE above before merge
    #       with no_outg_time_infos_df), then Utilities_df.consolidate_df will return an empty list (technically, an
    #       empty np.ndarray) for that trsf_pole_nb
    xfmr_for_PNs_df=Utilities_df.consolidate_df_OLD(
        df=xfmr_for_PNs_df, 
        groupby_col=df_mp_trsf_pole_nb_col, 
        cols_shared_by_group=[t_min_col, t_max_col], 
        cols_to_collect_in_lists=[return_prem_nbs_col]
    )    
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    for trsf_pole_nb_i, row_i in xfmr_for_PNs_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so much check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        if((not isinstance(PNs_i, list) and pd.isna(PNs_i)) or 
           len(PNs_i)==0):
            active_SNs_df_i = pd.DataFrame()
        else:
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=PNs_i,
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=dt_0_i,
                dt_1=dt_1_i,
                output_index=None,
                output_groupby=[df_mp_prem_nb_col], 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False
            )
            active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
        active_SNs_df_i[df_mp_trsf_pole_nb_col] = trsf_pole_nb_i
        active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
        assert(trsf_pole_nb_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[trsf_pole_nb_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    active_SNs_df = Utilities_df.consolidate_df_OLD(
        df=active_SNs_df, 
        groupby_col=df_mp_trsf_pole_nb_col, 
        cols_shared_by_group=None, 
        cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
        include_groupby_col_in_output_cols=False, 
        allow_duplicates_in_lists=False, 
        recover_uniqueness_violators=True, 
        rename_cols=None, 
        verbose=False
    )
    #-----
    # Change [nan] entries to []
    active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0, df_mp_serial_number_col] = [[]]
    active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0, df_mp_prem_nb_col] = [[]]
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Moved to OutageMdlrPrep
def add_xfmr_active_SNs_to_rcpo_df_OLD(
    rcpo_df, 
    set_xfmr_nSNs=True, 
    include_active_xfmr_PNs=False, #Should be equal to the PNs already in rcpo_df!
    df_mp_curr=None,
    df_mp_hist=None, 
    no_outg_time_infos_df=None, 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None, 
    xfmr_SNs_col='_xfmr_SNs', 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_PNs_col='_xfmr_PNs', 
    xfmr_nPNs_col='_xfmr_nPNs', 
):
    r"""
    NOTE: If include_active_xfmr_PNs is True, this column (named xfmr_PNs_col='_xfmr_SNs') should be 
          equal to the PNs already in rcpo_df!
    NOTE: xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col should all be strings, not tuples.
          If column is multiindex, the level_0 value will be handled below.
          
    NOTE: If any of xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col are already contained in 
          rcpo_df, they will be replaced.  This is needed so that the merge operation does not come back with _x and _y
          values.  So, one should make sure this function call is truly needed, as grabbing the serial numbers for the
          outages typically takes a couple/few minutes.
          
    NOTE: To make things run faster, the user can supply df_mp_curr and df_mp_hist.  These will be included in 
          get_active_SNs_for_xfmrs_kwargs.
          NOTE: If df_mp_curr/df_mp_hist is also supplied in addtnl_get_active_SNs_for_xfmrs_kwargs,
                that/those in addtnl_get_active_SNs_for_xfmrs_kwargs will ultimately be used (not the
                explicity df_mp_hist/curr in the function arguments!)
          CAREFUL: If one does supple df_mp_curr/hist, one must be certain these DFs contain all necessary elements!
    """
    #-------------------------
    get_active_SNs_for_xfmrs_kwargs = dict(
        trsf_pole_nbs=rcpo_df.index.unique().tolist(), 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        no_outg_time_infos_df=no_outg_time_infos_df, 
        return_prem_nbs_col=xfmr_PNs_col, 
        return_SNs_col=xfmr_SNs_col
    )
    if addtnl_get_active_SNs_for_xfmrs_kwargs is not None:
        get_active_SNs_for_xfmrs_kwargs = {**get_active_SNs_for_xfmrs_kwargs, 
                                           **addtnl_get_active_SNs_for_xfmrs_kwargs}
    active_SNs_df = get_active_SNs_for_xfmrs_OLD(**get_active_SNs_for_xfmrs_kwargs)
    assert(isinstance(active_SNs_df, pd.DataFrame))
    #-------------------------
    # Assert below might be too strong here...
    assert(sorted(rcpo_df.index.unique().tolist())==sorted(active_SNs_df.index.unique().tolist()))
    assert(rcpo_df.columns.nlevels<=2)
    if rcpo_df.columns.nlevels==1:
        #----------
        # See note above about columns being replaced/dropped
        cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
        if len(cols_to_drop)>0:
            rcpo_df = rcpo_df.drop(columns=cols_to_drop)
        #----------
        rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
        #----------
        if set_xfmr_nSNs:
            rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_SNs_col, xfmr_nSNs_col)
            if include_active_xfmr_PNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_PNs_col, xfmr_nPNs_col)
    else:
        # Currently, only expecting raw and/or norm.  No problem to allow more, but for now keep this to alert 
        # of anything unexpected
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        for i,level_0_val in enumerate(rcpo_df.columns.get_level_values(0).unique()):
            if i==0:
                active_SNs_df.columns = pd.MultiIndex.from_product([[level_0_val], active_SNs_df.columns])
            else:
                active_SNs_df.columns = active_SNs_df.columns.set_levels([level_0_val], level=0)
            #----------
            # See note above about columns being replaced/dropped
            cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
            if len(cols_to_drop)>0:
                rcpo_df = rcpo_df.drop(columns=cols_to_drop)
            #----------
            rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
            #----------
            if set_xfmr_nSNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_SNs_col), (level_0_val, xfmr_nSNs_col))
                if include_active_xfmr_PNs:
                    rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_PNs_col), (level_0_val, xfmr_nPNs_col))
    #-------------------------
    rcpo_df = rcpo_df.sort_index(axis=1,level=0)
    #-------------------------
    return rcpo_df

In [3]:
# MOVED to OutageMdlrPrep
def build_rcpo_df_norm_by_xfmr_active_nSNs_OLD(
    rcpo_df_raw, 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_SNs_col='_xfmr_SNs', 
    other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
    drop_xfmr_nSNs_eq_0=True, 
    new_level_0_val='counts_norm_by_xfmr_nSNs', 
    remove_SNs_cols=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    no_outg_time_infos_df=None, 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None
):
    r"""
    Build rcpo_df normalized by the number of serial numbers in each outage

    drop_xfmr_nSNs_eq_0:
      It is possible for the number of serial numbers in an outage to be zero!
      Premise numbers are always found, but the meter_premise database does not always 
        contain the premise numbers.
      Dividing by zero will make all counts for such an entry equal to NaN or inf.
      When drop_xfmr_nSNs_eq_0 is True, such entries will be removed.

    NOTE: xfmr_SNs_col and xfmr_nSNs_col should both be strings, not tuples.
          If column is MultiIndex, the level_0 value will be handled below.
    """
    #-------------------------
    n_counts_col = xfmr_nSNs_col
    list_col = xfmr_SNs_col
    #-------------------------
    # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects xfmr_SNs_col and xfmr_nSNs_col to be strings, not tuples
    #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
    assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
    assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
    #-----
    add_list_col_to_rcpo_df_func = add_xfmr_active_SNs_to_rcpo_df_OLD
    add_list_col_to_rcpo_df_kwargs = dict(
        set_xfmr_nSNs=True, 
        include_active_xfmr_PNs=True, 
        df_mp_curr=df_mp_curr,
        df_mp_hist=df_mp_hist, 
        no_outg_time_infos_df=no_outg_time_infos_df, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=addtnl_get_active_SNs_for_xfmrs_kwargs, 
        xfmr_SNs_col='_xfmr_SNs', 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_PNs_col='_xfmr_PNs', 
        xfmr_nPNs_col='_xfmr_nPNs', 
    )
    #-------------------------
    other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
    drop_n_counts_eq_0 = drop_xfmr_nSNs_eq_0
    new_level_0_val = new_level_0_val
    remove_ignored_cols = remove_SNs_cols
    #-------------------------
    return MECPODf.build_rcpo_df_norm_by_list_counts(
        rcpo_df_raw=rcpo_df_raw, 
        n_counts_col=n_counts_col, 
        list_col=list_col, 
        add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
        add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
        other_col_tags_to_ignore=other_col_tags_to_ignore, 
        drop_n_counts_eq_0=drop_n_counts_eq_0, 
        new_level_0_val=new_level_0_val, 
        remove_ignored_cols=remove_ignored_cols
    )

### NEW FUNCTIONS

In [ ]:
# Moved to OutageMdlrPrep
def reset_index_and_identify_cols_to_merge(
    df, 
    merge_on, 
    tag_for_idx_names=None
):
    r"""
    Designed to work with merge_rcpo_and_df (but should definitely be useful elsewhere), which 
    allows the user to join the DFs by columns, specific index levels or any mixture of the two.
    
    In order to achieve this type of general merging, it is easiest to call reset_index, making everything
      to be merged on a column.
    HOWEVER, in order to keep track of the original indices, which likely will be restored after the merge,
      it is important for all index levels to have a name.
      
    merge_on:
        This is a list that directs the columns/indices to be used in the join.
        Column identifiers:
            Single strings for normal DF, lists/tuples for MultiIndex columns
        Index identifiers:
            f'index_{idx_level}' to specify index level by number
            ('index', idx_level_name) to specify index level by name
            
    RETURNS:
        A dict object with keys = ['df', 'df_idx_names_OG', 'df_idx_names', 'reset_merge_on']
    """
    #-------------------------
    df=df.copy()
    #-------------------------
    assert(Utilities.is_object_one_of_types(merge_on, [list, tuple]))
    #-------------------------
    # Make sure all index levels have names!
    # If the level has a name, the code below will leave it unchanged in df_idx_names
    # If it does not have a name, it will be names f'index_{idx_level}', where idx_level 
    #   is the index level number
    df_idx_names_OG = list(df.index.names)
    df_idx_names = [x if x is not None else f'index_{i}' 
                    for i,x in enumerate(df_idx_names_OG)]
    if tag_for_idx_names is not None:
        df_idx_names = [f'{x}_{tag_for_idx_names}' for x in df_idx_names]
    df.index.names=df_idx_names
    #-------------------------
    reset_merge_on = []
    for idfr in merge_on:
        assert(Utilities.is_object_one_of_types(idfr, [str, list, tuple]))
        if idfr in df.columns:
            reset_merge_on.append(idfr)
        else:
            # Must be in indices!
            if isinstance(idfr, str):
                assert(idfr.startswith('index'))
                if idfr=='index':
                    idfr_idx_lvl=0
                else:
                    idfr_idx_lvl = re.findall('index_(\d*)', idfr)
                    assert(len(idfr_idx_lvl)==1)
                    idfr_idx_lvl=idfr_idx_lvl[0]
                    idfr_idx_lvl=int(idfr_idx_lvl)
            else:
                assert(len(idfr)==2)
                assert(idfr[0]=='index')
                idx_level_name = idfr[1]
                # If tag_for_idx_names, df.index.names already changed, so idx_level_name must be adjusted
                if tag_for_idx_names is not None:
                    idx_level_name = f'{idx_level_name}_{tag_for_idx_names}'
                assert(idx_level_name in df.index.names)
                idfr_idx_lvl = df.index.names.index(idx_level_name)
            #---------------
            assert(idfr_idx_lvl < df.index.nlevels)
            reset_merge_on_i = df.index.names[idfr_idx_lvl]
            # NOTE: If df.columns.nlevels>1, then calling df.reset_index() below
            #       will make the bottom level reset_merge_on_i and all the rest ''
            #       e.g., if nlevels=2, after df.reset_index(), reset_merge_on_i--> (reset_merge_on_i, '')
            if df.columns.nlevels>1:
                reset_merge_on_i=tuple([reset_merge_on_i] + ['']*(df.columns.nlevels-1))
            reset_merge_on.append(reset_merge_on_i)
    #-------------------------
    # Call reset_index on df, making all indices into columns, and double check that all reset_merge_on
    #   are contained in the columns
    df = df.reset_index()
    assert(len(set(reset_merge_on).difference(set(df.columns)))==0)
    #-------------------------
    return dict(
        df=df, 
        df_idx_names_OG=df_idx_names_OG, 
        df_idx_names=df_idx_names, 
        reset_merge_on=reset_merge_on
    )

# Moved to OutageMdlrPrep
def merge_rcpo_and_df(
    rcpo_df, 
    df_2, 
    rcpo_df_on,
    df_2_on, 
    how='left'
):
    r"""
    Merge together rcpo_df and df_2 dfs.
    Designed specifically for rcpo_df and time_infos_df.
    
    rcpo_df_on/df_2_on:
        These are lists which direct the columns/indices to be used in the join.
        Column identifiers:
            Single strings for normal DF, lists/tuples for MultiIndex columns
        Index identifiers:
            f'index_{idx_level}' to specify index level by number
            ('index', idx_level_name) to specify index level by name
            
        NOTE: Calling reset_index() on both DFs will be the easiest method for this type of general merging, in
              which the DFs can be merged by columns, specific index levels or any mixture of the two.
              This is done through the use of reset_index_and_identify_cols_to_merge
    """
    #--------------------------------------------------
    # Only expecting at most 2 levels in columns (e.g., counts or counts_norm as level 0, and reason as level 1)
    #     - probably not a necessary assertion, if function to be expanded later
    #   Ultimately, the number of levels in df_2 will match that of rcpo_df, so making the same
    #     assertion on df_2
    assert((rcpo_df.columns.nlevels <= 2) and (df_2.columns.nlevels <= 2))
    if rcpo_df.columns.nlevels==2:
        # If rcpo_df has two column levels, df_2 must also for the proper merging to occur
        #   --Merging with unequal levels will cause all levels to be collapsed down to single dimension
        #-----
        # Expecting at most 2 unique values for level 0 (definitely not a necessary assertion)
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        # In this case, likely df_2 has only single level columns.
        #   For proper merge, the number of levels should match
        if df_2.columns.nlevels==1:
            level_0_vals = rcpo_df.columns.get_level_values(0).unique().tolist()
            # Add new top level to df_2 with value equal to level_0_vals[0]
            df_2=Utilities_df.prepend_level_to_MultiIndex(
                df=df_2, level_val=level_0_vals[0], level_name=None, axis=1
            )
            if len(level_0_vals)>1:
                # Grab df without new column level to be copied to other new column level values
                # NOTE: If extra [] placed around level_0_vals[0] below, the new column level would be returned
                #       (which is not desired here!)
                df_0 = df_2[level_0_vals[0]]
                df_0_cols = df_0.columns.tolist()
                # Reproduce the entries of df_2 for all column level 0 values in rcpo_df
                for i_lvl in range(1,len(level_0_vals)):
                    new_cols = pd.MultiIndex.from_product([[level_0_vals[i_lvl]], df_0_cols])
                    df_2[new_cols] = df_0.copy()
        # Now, at this stage, rcpo_df and df_2 should have the same number of column levels
        #   and should have overlapping level 0 values if nlevels>1
        assert((rcpo_df.columns.nlevels <= 2) and (df_2.columns.nlevels <= 2)) #not needed
        assert(rcpo_df.columns.nlevels == df_2.columns.nlevels)
        if rcpo_df.columns.nlevels == 2:
            # Make sure overlapping 0 values.  
            # I suppose user could supply df_2 with only a single level 0 value while rcpo_df has
            #   two values.  In this case, only the single value would be merged to rcpo_df
            assert(len(set(df_2.columns.get_level_values(0).unique().tolist()).difference(
                set(rcpo_df.columns.get_level_values(0).unique().tolist())
            ))==0)
    else:
        assert(rcpo_df.columns.nlevels==df_2.columns.nlevels==1)
    #--------------------------------------------------
    reset_rcpo_df_dict = reset_index_and_identify_cols_to_merge(
        df=rcpo_df, 
        merge_on=rcpo_df_on, 
        tag_for_idx_names='from_rcpo_df'
    )

    reset_df_2_dict = reset_index_and_identify_cols_to_merge(
        df=df_2, 
        merge_on=df_2_on, 
        tag_for_idx_names='from_df_2'
    )
    #-------------------------
    merged_df = pd.merge(
        reset_rcpo_df_dict['df'], 
        reset_df_2_dict['df'], 
        left_on=reset_rcpo_df_dict['reset_merge_on'], 
        right_on = reset_df_2_dict['reset_merge_on'], 
        how=how
    )
    #-------------------------
    # Want to set index first before changing names back to originals, as the
    #   originals could have been None
    merged_df = merged_df.set_index(reset_rcpo_df_dict['df_idx_names'])

    # Two lines below (instead of calling simply df.index.names=df_idx_names_OG) 
    #   ensures the order is the same
    rcpo_df_rename_dict = dict(zip(reset_rcpo_df_dict['df_idx_names'], reset_rcpo_df_dict['df_idx_names_OG']))
    merged_df.index.names = [rcpo_df_rename_dict[x] for x in merged_df.index.names]
    #-------------------------
    # When merging two columns whose names are different, both columns are kept
    #   This is redundant, as these will have identical values, as they were merged, so get rid of
    #   Note: If the column names are the same, this is obviously not an issue (hence the need to find
    #         cols_to_drop below, instead of simply dropping all of reset_df_2_dict['reset_merge_on']
    cols_to_drop = [x for x in reset_df_2_dict['reset_merge_on'] if x in merged_df.columns]
    merged_df = merged_df.drop(columns=cols_to_drop)

    # Rename columns from df_2 to original values (if original values were not None!)
    df_2_rename_dict = dict(zip(reset_df_2_dict['df_idx_names'], reset_df_2_dict['df_idx_names_OG']))
    df_2_rename_dict = {k:v for k,v in df_2_rename_dict.items() if k in merged_df.columns and v is not None}
    merged_df=merged_df.rename(columns=df_2_rename_dict)
    #-------------------------
    return merged_df

In [ ]:
# TODO still needs work...
# This replaces get_active_SNs_for_xfmrs_OLD (but, typically use get_active_SNs_for_xfmrs_in_rcpo_df)
# Moved to OutageMdlrPrep
def get_active_SNs_for_xfmrs(
    trsf_pole_nbs,     
    df_mp_curr, 
    df_mp_hist,
    time_infos_df,     
    time_infos_to_PNs = ['index'], 
    PNs_to_time_infos = ['index'], 
    how='left',     
    output_trsf_pole_nb_col=None, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
      
    time_infos_to_PNs:
      Defines how time_infos_df and PNs_for_xfmrs will be merged.
      NOTE: PNs_for_xfmrs will have indices equal to trsf_pole_nbs and values equal to lists of associated prem_nbs
      See merge_rcpo_and_df and reset_index_and_identify_cols_to_merge for more information

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #-----
    # Remove any duplicates from time_infos_df
    if time_infos_df.index.nlevels==1:
        tmp_col = Utilities.generate_random_string()
        time_infos_df[tmp_col] = time_infos_df.index
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=[tmp_col])
    else:
        tmp_cols = [Utilities.generate_random_string() for _ in range(time_infos_df.index.nlevels)]
        for i_col, tmp_col in enumerate(tmp_cols):
            time_infos_df[tmp_col] = time_infos_df.index.get_level_values(i_col)
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=tmp_cols)
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together time_infos_df and PNs_for_xfmrs
    #-----
    time_infos_df = merge_rcpo_and_df(
        rcpo_df=time_infos_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=time_infos_to_PNs,
        df_2_on=PNs_to_time_infos, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}
    if output_trsf_pole_nb_col is None:
        output_trsf_pole_nb_col='trsf_pole_nb'
    for trsf_pole_nb in trsf_pole_nbs:
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        PNs_i=time_infos_df.loc[trsf_pole_nb, return_prem_nbs_col]
        dt_0_i=time_infos_df.loc[trsf_pole_nb, t_min_col]
        dt_1_i=time_infos_df.loc[trsf_pole_nb, t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check time_infos_df.loc[trsf_pole_nb, t_min_col] instead!
            assert(Utilities.is_object_one_of_types(time_infos_df.loc[trsf_pole_nb, t_min_col], [list, np.ndarray]) and 
                   len(time_infos_df.loc[trsf_pole_nb, t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        if((not isinstance(PNs_i, list) and pd.isna(PNs_i)) or 
           len(PNs_i)==0):
            active_SNs_df_i = pd.DataFrame()
        else:
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=PNs_i,
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=dt_0_i,
                dt_1=dt_1_i,
                output_index=None,
                output_groupby=[df_mp_prem_nb_col], 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False
            )
            active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
            active_SNs_df_i[output_trsf_pole_nb_col] = trsf_pole_nb
            active_SNs_df_i = active_SNs_df_i.set_index(output_trsf_pole_nb_col)
        else:
            active_SNs_df_i[output_trsf_pole_nb_col] = trsf_pole_nb
            active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
            active_SNs_df_i = Utilities_df.consolidate_df(
                df=active_SNs_df_i, 
                groupby_cols=[output_trsf_pole_nb_col], 
                cols_shared_by_group=None, 
                cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
                include_groupby_cols_in_output_cols=False, 
                allow_duplicates_in_lists=False, 
                recover_uniqueness_violators=True, 
                rename_cols=None, 
                verbose=False
            )
        assert(trsf_pole_nb not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[trsf_pole_nb] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    # Change [nan] entries to []
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col] = active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col].apply(lambda x: [])
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col] = active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col].apply(lambda x: [])
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Replaces get_active_SNs_for_xfmrs_OLD, but should probably build get_active_SNs_for_xfmrs
#  which accepts a list of trsf_pole_nbs instead of rcpo_df, which this function can use
# Moved to OutageMdlrPrep
def get_active_SNs_for_xfmrs_in_rcpo_df(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    df_mp_curr, 
    df_mp_hist,
    time_infos_df, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    The trsf_pole_nbs should be contained in rcpo_df, and will be found using the trsf_pole_nbs_loc
      parameter described below.
      
    trsf_pole_nbs_loc:
        Directs where the transformer pole numbers are located
        This should identify an index (w/ level)
        Set equal to 'index' for normal DFs, or when trsf_pole_nbs are in level 0 of index.
        For a DF with MultiIndex index, there are two options:
            i.  Set equal to f'index_{idx_level}' for a DF with MutliIndex index, where idx_level
                is an int identifying the level in which the trsf_pole_nbs reside
            ii. Set equal to the tuple ('index', trsf_pole_nbs_idx_name), where trsf_pole_nbs_idx_name is
            the name of the index level in which the trsf_pole_nbs reside.

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #-----
    # Remove any duplicates from time_infos_df
    if time_infos_df.index.nlevels==1:
        tmp_col = Utilities.generate_random_string()
        time_infos_df[tmp_col] = time_infos_df.index
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=[tmp_col])
    else:
        tmp_cols = [Utilities.generate_random_string() for _ in range(time_infos_df.index.nlevels)]
        for i_col, tmp_col in enumerate(tmp_cols):
            time_infos_df[tmp_col] = time_infos_df.index.get_level_values(i_col)
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=tmp_cols)
    #--------------------------------------------------
    # trsf_pole_nbs_loc can be a string or tuple/list
    # First, find trsf_pole_nbs and trsf_pole_nbs_idx_lvl
    assert(Utilities.is_object_one_of_types(trsf_pole_nbs_loc, [str, list, tuple]))
    if isinstance(trsf_pole_nbs_loc, str):
        assert(trsf_pole_nbs_loc.startswith('index'))
        if trsf_pole_nbs_loc=='index':
            trsf_pole_nbs_idx_lvl = 0
        else:
            trsf_pole_nbs_idx_lvl = re.findall('index_(\d*)', trsf_pole_nbs_loc)
            assert(len(trsf_pole_nbs_idx_lvl)==1)
            trsf_pole_nbs_idx_lvl=trsf_pole_nbs_idx_lvl[0]
            trsf_pole_nbs_idx_lvl=int(trsf_pole_nbs_idx_lvl)
    else:
        assert(len(trsf_pole_nbs_loc)==2)
        assert(trsf_pole_nbs_loc[0]=='index')
        assert(trsf_pole_nbs_loc[1] in rcpo_df.index.names)
        trsf_pole_nbs_idx_lvl = rcpo_df.index.names.index(trsf_pole_nbs_loc[1])
        #---------------
        assert(trsf_pole_nbs_idx_lvl < rcpo_df.index.nlevels)
        trsf_pole_nbs = rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist()
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together rcpo_df, time_infos_df and PNs_for_xfmrs
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=time_infos_df, 
        rcpo_df_on=rcpo_df_to_time_infos_on,
        df_2_on=time_infos_to_rcpo_df_on, 
        how=how
    )
    #-----
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=rcpo_df_to_PNs_on,
        df_2_on=PNs_to_rcpo_df_on, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    rcpo_idx_names = list(rcpo_df.index.names)
    assert(not any([x is None for x in rcpo_idx_names]))
    for idx_i, row_i in rcpo_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        # Purpose of making idx_names_w_vals a list of tuples, instead of a dict, is to ensure the correct order is maintained
        #   Dicts usually return the correct order, but this is not guaranteed
        if len(rcpo_idx_names)==1:
            assert(rcpo_df.index.nlevels==1)
            idx_names_w_vals = [(rcpo_idx_names[0], idx_i)]
        else:
            idx_names_w_vals = [((rcpo_idx_names[i] if i!=trsf_pole_nbs_idx_lvl else df_mp_trsf_pole_nb_col), idx_i[i]) 
                                for i in range(len(idx_i))]
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        if((not isinstance(PNs_i, list) and pd.isna(PNs_i)) or 
           len(PNs_i)==0):
            active_SNs_df_i = pd.DataFrame()
        else:
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=PNs_i,
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=dt_0_i,
                dt_1=dt_1_i,
                output_index=None,
                output_groupby=[df_mp_prem_nb_col], 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False
            )
            active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
            for name,val in idx_names_w_vals:
                active_SNs_df_i[name] = val
            active_SNs_df_i = active_SNs_df_i.set_index([x[0] for x in idx_names_w_vals])
        else:
            for name,val in idx_names_w_vals:
                active_SNs_df_i[name] = val
            active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
            active_SNs_df_i = Utilities_df.consolidate_df(
                df=active_SNs_df_i, 
                groupby_cols=[x[0] for x in idx_names_w_vals], 
                cols_shared_by_group=None, 
                cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
                include_groupby_cols_in_output_cols=False, 
                allow_duplicates_in_lists=False, 
                recover_uniqueness_violators=True, 
                rename_cols=None, 
                verbose=False
            )
        assert(idx_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[idx_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    # Change [nan] entries to []
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col] = active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col].apply(lambda x: [])
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col] = active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col].apply(lambda x: [])
    #-------------------------
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# act_mfs = get_active_SNs_for_xfmrs_in_rcpo_df(
#     rcpo_df=rcpo_df_raw, 
#     trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
#     df_mp_curr=mp_df_curr_hist['mp_df_curr'],
#     df_mp_hist=mp_df_curr_hist['mp_df_hist'], 
#     time_infos_df=time_infos_df, 
#     rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
#     time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
#     how=how, 
#     rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
#     PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
#     assert_all_trsf_pole_nbs_found=False
# )

In [ ]:
# Replaces get_active_SNs_for_xfmrs_OLD, but should probably build get_active_SNs_for_xfmrs
#  which accepts a list of trsf_pole_nbs instead of rcpo_df, which this function can use
# Moved to OutageMdlrPrep
def get_active_SNs_for_xfmrs_in_rcpo_df_v2(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    df_mp_curr, 
    df_mp_hist,
    time_infos_df, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    return_SNs_col='SNs', 
    return_prem_nbs_col='prem_nbs', 
    assert_all_trsf_pole_nbs_found=True, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    t_min_col='t_min', 
    t_max_col='t_max'
):
    r"""
    Difficulty is that default.meter_premise_hist does not have trsf_pole_nb field.
    Therefore, one must use default.meter_premise to find the premise numbers for xfrms in trsf_pole_nbs,
      then use those PNs to select the correct entries from default.meter_premise_hist.
    The trsf_pole_nbs should be contained in rcpo_df, and will be found using the trsf_pole_nbs_loc
      parameter described below.
      
    trsf_pole_nbs_loc:
        Directs where the transformer pole numbers are located
        This should identify an index (w/ level)
        Set equal to 'index' for normal DFs, or when trsf_pole_nbs are in level 0 of index.
        For a DF with MultiIndex index, there are two options:
            i.  Set equal to f'index_{idx_level}' for a DF with MutliIndex index, where idx_level
                is an int identifying the level in which the trsf_pole_nbs reside
            ii. Set equal to the tuple ('index', trsf_pole_nbs_idx_name), where trsf_pole_nbs_idx_name is
            the name of the index level in which the trsf_pole_nbs reside.

    If df_mp_curr OR df_mp_hist is not supplied, both will be built!
    
    addtnl_mp_df_curr_cols/addtnl_mp_df_hist_cols:
      Only used when df_mp_curr/df_mp_hist not supplied and therefore need to be built
      
    """
    #--------------------------------------------------
    assert(t_min_col in time_infos_df.columns and 
           t_max_col in time_infos_df.columns)
    time_infos_df = time_infos_df[[t_min_col, t_max_col]]
    #-----
    # Remove any duplicates from time_infos_df
    if time_infos_df.index.nlevels==1:
        tmp_col = Utilities.generate_random_string()
        time_infos_df[tmp_col] = time_infos_df.index
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=[tmp_col])
    else:
        tmp_cols = [Utilities.generate_random_string() for _ in range(time_infos_df.index.nlevels)]
        for i_col, tmp_col in enumerate(tmp_cols):
            time_infos_df[tmp_col] = time_infos_df.index.get_level_values(i_col)
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=tmp_cols)
    #--------------------------------------------------
    # trsf_pole_nbs_loc can be a string or tuple/list
    # First, find trsf_pole_nbs and trsf_pole_nbs_idx_lvl
    assert(Utilities.is_object_one_of_types(trsf_pole_nbs_loc, [str, list, tuple]))
    if isinstance(trsf_pole_nbs_loc, str):
        assert(trsf_pole_nbs_loc.startswith('index'))
        if trsf_pole_nbs_loc=='index':
            trsf_pole_nbs_idx_lvl = 0
        else:
            trsf_pole_nbs_idx_lvl = re.findall('index_(\d*)', trsf_pole_nbs_loc)
            assert(len(trsf_pole_nbs_idx_lvl)==1)
            trsf_pole_nbs_idx_lvl=trsf_pole_nbs_idx_lvl[0]
            trsf_pole_nbs_idx_lvl=int(trsf_pole_nbs_idx_lvl)
    else:
        assert(len(trsf_pole_nbs_loc)==2)
        assert(trsf_pole_nbs_loc[0]=='index')
        assert(trsf_pole_nbs_loc[1] in rcpo_df.index.names)
        trsf_pole_nbs_idx_lvl = rcpo_df.index.names.index(trsf_pole_nbs_loc[1])
        #---------------
        assert(trsf_pole_nbs_idx_lvl < rcpo_df.index.nlevels)
        trsf_pole_nbs = rcpo_df.index.get_level_values(trsf_pole_nbs_idx_lvl).tolist()
    #--------------------------------------------------
    #-------------------------
    necessary_mp_cols = [df_mp_serial_number_col, df_mp_prem_nb_col, df_mp_install_time_col, df_mp_removal_time_col]
    #-------------------------
    if df_mp_curr is None or df_mp_hist is None:
        mp_df_curr_hist = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
            trsf_pole_nbs, 
            join_curr_hist=False, 
            addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
            addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
            df_mp_serial_number_col=df_mp_serial_number_col, 
            df_mp_prem_nb_col=df_mp_prem_nb_col, 
            df_mp_install_time_col=df_mp_install_time_col, 
            df_mp_removal_time_col=df_mp_removal_time_col, 
            df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
        )
        df_mp_curr = mp_df_curr_hist['mp_df_curr']
        df_mp_hist = mp_df_curr_hist['mp_df_hist']
    #-------------------------
    # At a bare minimum, df_mp_curr and df_mp_hist must both have the following columns:
    #   necessary_mp_cols = ['mfr_devc_ser_nbr', 'prem_nb', 'inst_ts', 'rmvl_ts']
    assert(all([x in df_mp_curr.columns for x in necessary_mp_cols+[df_mp_trsf_pole_nb_col]]))
    assert(all([x in df_mp_hist.columns for x in necessary_mp_cols]))
    #-------------------------
    # PNs_for_xfmrs is a DF with trsf_pole_nbs indices and elements which are lists of PNs for each xfmr
    PNs_for_xfmrs = MeterPremise.get_SNs_andor_PNs_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        include_SNs=False,
        include_PNs=True,
        trsf_pole_nb_col=df_mp_trsf_pole_nb_col, 
        serial_number_col=df_mp_serial_number_col, 
        prem_nb_col=df_mp_prem_nb_col, 
        return_SNs_col=None, #Not grabbing SNs
        return_PNs_col=return_prem_nbs_col, 
        assert_all_trsf_pole_nbs_found=assert_all_trsf_pole_nbs_found, 
        mp_df=df_mp_curr, 
        return_mp_df_also=False
    )
    #-------------------------
    # Join together rcpo_df, time_infos_df and PNs_for_xfmrs
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=time_infos_df, 
        rcpo_df_on=rcpo_df_to_time_infos_on,
        df_2_on=time_infos_to_rcpo_df_on, 
        how=how
    )
    #-----
    rcpo_df = merge_rcpo_and_df(
        rcpo_df=rcpo_df, 
        df_2=PNs_for_xfmrs, 
        rcpo_df_on=rcpo_df_to_PNs_on,
        df_2_on=PNs_to_rcpo_df_on, 
        how=how
    )
    #--------------------------------------------------
    # Only reason for making dict is to ensure trsf_pole_nbs are not repeated 
    active_SNs_in_xfmrs_dfs_dict = {}

    rcpo_idx_names = list(rcpo_df.index.names)
    assert(not any([x is None for x in rcpo_idx_names]))
    for idx_i, row_i in rcpo_df.iterrows():
        # active_SNs_df_i will have indices equal to premise numbers and value equal to lists
        #   of active SNs for each PN
        # Purpose of making idx_names_w_vals a list of tuples, instead of a dict, is to ensure the correct order is maintained
        #   Dicts usually return the correct order, but this is not guaranteed
        if len(rcpo_idx_names)==1:
            assert(rcpo_df.index.nlevels==1)
            idx_names_w_vals = [(rcpo_idx_names[0], idx_i)]
        else:
            idx_names_w_vals = [((rcpo_idx_names[i] if i!=trsf_pole_nbs_idx_lvl else df_mp_trsf_pole_nb_col), idx_i[i]) 
                                for i in range(len(idx_i))]
        PNs_i=row_i[return_prem_nbs_col]
        dt_0_i=row_i[t_min_col]
        dt_1_i=row_i[t_max_col]
        #-----
        # See NOTEs above regarding t_min/t_max being empty
        # In such a case, it is simply impossibe (with the summary files currently generated) to access
        #   the date over which the data would have been run, if any events existed.
        #   In future versions, this information will be included in the summary files!
        # I don't want to completely exclude these (by e.g., setting dt_0_i=pd.Timestamp.min and 
        #   dt_1_i=pd.Timestamp.max), so I will simply include the meters which are active TODAY.
        # This obviously is not correct, but this occurrence is rare (only happening when every single meter
        #   on a transformer had no events during the time period) and this crude approximation will be fine.
        if Utilities.is_object_one_of_types(dt_0_i, [list, np.ndarray]):
            assert(len(dt_0_i)==0)
            # I believe if this happens for one it should happen for both...
            assert(Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]) and len(dt_1_i)==0)
            dt_0_i=pd.Timestamp.today()
        if Utilities.is_object_one_of_types(dt_1_i, [list, np.ndarray]):
            assert(len(dt_1_i)==0)
            # I believe if this happens for one it should happen for both...
            # But, dt_0_i changed already above, so must check row_i[t_min_col] instead!
            assert(Utilities.is_object_one_of_types(row_i[t_min_col], [list, np.ndarray]) and len(row_i[t_min_col])==0)
            dt_1_i=pd.Timestamp.today()
        if((not isinstance(PNs_i, list) and pd.isna(PNs_i)) or 
           len(PNs_i)==0):
            active_SNs_df_i = pd.DataFrame()
        else:
            active_SNs_df_i = MeterPremise.get_active_SNs_for_PNs_at_datetime_interval(
                PNs=PNs_i,
                df_mp_curr=df_mp_curr, 
                df_mp_hist=df_mp_hist, 
                dt_0=dt_0_i,
                dt_1=dt_1_i,
                output_index=None,
                output_groupby=[df_mp_prem_nb_col], 
                include_prems_wo_active_SNs_when_groupby=True, 
                assert_all_PNs_found=False
            )
            active_SNs_df_i=active_SNs_df_i.reset_index()
        if active_SNs_df_i.shape[0]==0:
            active_SNs_df_i[df_mp_prem_nb_col] = np.nan
            active_SNs_df_i[df_mp_serial_number_col] = [[]] 
        for name,val in idx_names_w_vals:
            active_SNs_df_i[name] = val
        active_SNs_df_i = active_SNs_df_i.explode(df_mp_serial_number_col)
        assert(idx_i not in active_SNs_in_xfmrs_dfs_dict)
        active_SNs_in_xfmrs_dfs_dict[idx_i] = active_SNs_df_i
    #-------------------------
    active_SNs_df = pd.concat(list(active_SNs_in_xfmrs_dfs_dict.values()))
    #-------------------------
    active_SNs_df = Utilities_df.consolidate_df(
        df=active_SNs_df, 
        groupby_cols=[x[0] for x in idx_names_w_vals], 
        cols_shared_by_group=None, 
        cols_to_collect_in_lists=[df_mp_serial_number_col, df_mp_prem_nb_col], 
        include_groupby_cols_in_output_cols=False, 
        allow_duplicates_in_lists=False, 
        recover_uniqueness_violators=True, 
        rename_cols=None, 
        verbose=False
    )
    #-------------------------
    # Change [nan] entries to []
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col] = active_SNs_df.loc[active_SNs_df[df_mp_serial_number_col].isna(), df_mp_serial_number_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_serial_number_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col] = active_SNs_df.loc[found_nans_srs, df_mp_serial_number_col].apply(lambda x: [])
    #-----
    # First, if any entries equal NaN, change to []
    active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col] = active_SNs_df.loc[active_SNs_df[df_mp_prem_nb_col].isna(), df_mp_prem_nb_col].apply(lambda x: [])
    # Now, change any entries equal to [] or [NaN] to []
    found_nans_srs = active_SNs_df[df_mp_prem_nb_col].apply(lambda x: len([ix for ix in x if not pd.isna(ix)]))==0
    if found_nans_srs.sum()>0:
        active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col] = active_SNs_df.loc[found_nans_srs, df_mp_prem_nb_col].apply(lambda x: [])
    #------------------------- 
    active_SNs_df = active_SNs_df.rename(columns={
        df_mp_prem_nb_col:return_prem_nbs_col, 
        df_mp_serial_number_col:return_SNs_col
    })
    #-------------------------
    return active_SNs_df

In [ ]:
# Replaces add_xfmr_active_SNs_to_rcpo_df_OLD
# Moved to OutageMdlrPrep
def add_xfmr_active_SNs_to_rcpo_df(
    rcpo_df, 
    trsf_pole_nbs_loc, 
    set_xfmr_nSNs=True, 
    include_active_xfmr_PNs=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    time_infos_df=None, 
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None, 
    xfmr_SNs_col='_xfmr_SNs', 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_PNs_col='_xfmr_PNs', 
    xfmr_nPNs_col='_xfmr_nPNs', 
):
    r"""
    NOTE: If include_active_xfmr_PNs is True, this column (named xfmr_PNs_col='_xfmr_SNs') should be 
          equal to the PNs already in rcpo_df!
    NOTE: xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col should all be strings, not tuples.
          If column is multiindex, the level_0 value will be handled below.
          
    NOTE: If any of xfmr_SNs_col, xfmr_nSNs_col, xfmr_PNs_col, and xfmr_nPNs_col are already contained in 
          rcpo_df, they will be replaced.  This is needed so that the merge operation does not come back with _x and _y
          values.  So, one should make sure this function call is truly needed, as grabbing the serial numbers for the
          outages typically takes a couple/few minutes.
          
    NOTE: To make things run faster, the user can supply df_mp_curr and df_mp_hist.  These will be included in 
          get_active_SNs_for_xfmrs_kwargs.
          NOTE: If df_mp_curr/df_mp_hist is also supplied in addtnl_get_active_SNs_for_xfmrs_kwargs,
                that/those in addtnl_get_active_SNs_for_xfmrs_kwargs will ultimately be used (not the
                explicity df_mp_hist/curr in the function arguments!)
          CAREFUL: If one does supple df_mp_curr/hist, one must be certain these DFs contain all necessary elements!
    """
    #-------------------------
    get_active_SNs_for_xfmrs_kwargs = dict(
        rcpo_df=rcpo_df, 
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        time_infos_df=time_infos_df, 
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        return_prem_nbs_col=xfmr_PNs_col, 
        return_SNs_col=xfmr_SNs_col
    )
    if addtnl_get_active_SNs_for_xfmrs_kwargs is not None:
        get_active_SNs_for_xfmrs_kwargs = {**get_active_SNs_for_xfmrs_kwargs, 
                                           **addtnl_get_active_SNs_for_xfmrs_kwargs}
    active_SNs_df = get_active_SNs_for_xfmrs_in_rcpo_df(**get_active_SNs_for_xfmrs_kwargs)
    assert(isinstance(active_SNs_df, pd.DataFrame))
    #-------------------------
    # Assert below might be too strong here...
    assert(sorted(rcpo_df.index.unique().tolist())==sorted(active_SNs_df.index.unique().tolist()))
    assert(rcpo_df.columns.nlevels<=2)
    if rcpo_df.columns.nlevels==1:
        #----------
        # See note above about columns being replaced/dropped
        cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
        if len(cols_to_drop)>0:
            rcpo_df = rcpo_df.drop(columns=cols_to_drop)
        #----------
        rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
        #----------
        if set_xfmr_nSNs:
            rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_SNs_col, xfmr_nSNs_col)
            if include_active_xfmr_PNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, xfmr_PNs_col, xfmr_nPNs_col)
    else:
        # Currently, only expecting raw and/or norm.  No problem to allow more, but for now keep this to alert 
        # of anything unexpected
        assert(rcpo_df.columns.get_level_values(0).nunique()<=2)
        for i,level_0_val in enumerate(rcpo_df.columns.get_level_values(0).unique()):
            if i==0:
                active_SNs_df.columns = pd.MultiIndex.from_product([[level_0_val], active_SNs_df.columns])
            else:
                active_SNs_df.columns = active_SNs_df.columns.set_levels([level_0_val], level=0)
            #----------
            # See note above about columns being replaced/dropped
            cols_to_drop = [x for x in rcpo_df.columns if x in active_SNs_df.columns]
            if len(cols_to_drop)>0:
                rcpo_df = rcpo_df.drop(columns=cols_to_drop)
            #----------
            rcpo_df = rcpo_df.merge(active_SNs_df, left_index=True, right_index=True)
            #----------
            if set_xfmr_nSNs:
                rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_SNs_col), (level_0_val, xfmr_nSNs_col))
                if include_active_xfmr_PNs:
                    rcpo_df = MECPODf.set_nSNs_from_SNs_in_rcpo_df(rcpo_df, (level_0_val, xfmr_PNs_col), (level_0_val, xfmr_nPNs_col))
    #-------------------------
    rcpo_df = rcpo_df.sort_index(axis=1,level=0)
    #-------------------------
    return rcpo_df

In [ ]:
# Replaces build_rcpo_df_norm_by_xfmr_active_nSNs_OLD
# Moved to OutageMdlrPrep
def build_rcpo_df_norm_by_xfmr_active_nSNs(
    rcpo_df_raw, 
    trsf_pole_nbs_loc, 
    xfmr_nSNs_col='_xfmr_nSNs', 
    xfmr_SNs_col='_xfmr_SNs', 
    other_SNs_col_tags_to_ignore=['_SNs', '_nSNs', '_prem_nbs', '_nprem_nbs', '_xfmr_PNs', '_xfmr_nPNs'], 
    drop_xfmr_nSNs_eq_0=True, 
    new_level_0_val='counts_norm_by_xfmr_nSNs', 
    remove_SNs_cols=False, 
    df_mp_curr=None,
    df_mp_hist=None, 
    time_infos_df=None,
    rcpo_df_to_time_infos_on = [('index', 'outg_rec_nb')], 
    time_infos_to_rcpo_df_on = ['index'], 
    how='left', 
    rcpo_df_to_PNs_on = [('index', 'trsf_pole_nb')], 
    PNs_to_rcpo_df_on = ['index'], 
    addtnl_get_active_SNs_for_xfmrs_kwargs=None
):
    r"""
    Build rcpo_df normalized by the number of serial numbers in each outage

    drop_xfmr_nSNs_eq_0:
      It is possible for the number of serial numbers in an outage to be zero!
      Premise numbers are always found, but the meter_premise database does not always 
        contain the premise numbers.
      Dividing by zero will make all counts for such an entry equal to NaN or inf.
      When drop_xfmr_nSNs_eq_0 is True, such entries will be removed.

    NOTE: xfmr_SNs_col and xfmr_nSNs_col should both be strings, not tuples.
          If column is MultiIndex, the level_0 value will be handled below.
    """
    #-------------------------
    n_counts_col = xfmr_nSNs_col
    list_col = xfmr_SNs_col
    #-------------------------
    # NOTE: MECPODf.add_outage_SNs_to_rcpo_df expects xfmr_SNs_col and xfmr_nSNs_col to be strings, not tuples
    #       as it handles the level 0 values if they exist.  So, if tuples, use only highest level values (i.e., level 1)
    assert(Utilities.is_object_one_of_types(list_col, [str, tuple]))
    assert(Utilities.is_object_one_of_types(n_counts_col, [str, tuple]))
    #-----
    add_list_col_to_rcpo_df_func = add_xfmr_active_SNs_to_rcpo_df
    add_list_col_to_rcpo_df_kwargs = dict(
        trsf_pole_nbs_loc=trsf_pole_nbs_loc, 
        set_xfmr_nSNs=True, 
        include_active_xfmr_PNs=True, 
        df_mp_curr=df_mp_curr,
        df_mp_hist=df_mp_hist, 
        time_infos_df=time_infos_df, 
        rcpo_df_to_time_infos_on=rcpo_df_to_time_infos_on, 
        time_infos_to_rcpo_df_on=time_infos_to_rcpo_df_on, 
        how=how, 
        rcpo_df_to_PNs_on=rcpo_df_to_PNs_on, 
        PNs_to_rcpo_df_on=PNs_to_rcpo_df_on, 
        addtnl_get_active_SNs_for_xfmrs_kwargs=addtnl_get_active_SNs_for_xfmrs_kwargs, 
        xfmr_SNs_col='_xfmr_SNs', 
        xfmr_nSNs_col='_xfmr_nSNs', 
        xfmr_PNs_col='_xfmr_PNs', 
        xfmr_nPNs_col='_xfmr_nPNs', 
    )
    #-------------------------
    other_col_tags_to_ignore = other_SNs_col_tags_to_ignore
    drop_n_counts_eq_0 = drop_xfmr_nSNs_eq_0
    new_level_0_val = new_level_0_val
    remove_ignored_cols = remove_SNs_cols
    #-------------------------
    return MECPODf.build_rcpo_df_norm_by_list_counts(
        rcpo_df_raw=rcpo_df_raw, 
        n_counts_col=n_counts_col, 
        list_col=list_col, 
        add_list_col_to_rcpo_df_func=add_list_col_to_rcpo_df_func, 
        add_list_col_to_rcpo_df_kwargs=add_list_col_to_rcpo_df_kwargs, 
        other_col_tags_to_ignore=other_col_tags_to_ignore, 
        drop_n_counts_eq_0=drop_n_counts_eq_0, 
        new_level_0_val=new_level_0_val, 
        remove_ignored_cols=remove_ignored_cols
    )

In [ ]:
# Moved to OutageMdlrPrep
def get_outg_time_infos_df(
    rcpo_df, 
    outg_rec_nb_idx_lvl, 
    times_relative_to_off_ts_only=True, 
    td_for_left=None, 
    td_for_right=None
):
    r"""
    Given a rcpo_df, with outg_rec_nbs stored in the index (located by outg_rec_nb_idx_lvl), construct the
    time_infos_df whose indices are outg_rec_nbs and columns, ['t_min', 't_max'], give the search time windows
    for each outage.
    
    This was designed, and is generally used, for finding the active meters for each outage.
    
    td_for_left / td_for_right: 
        datetime.timedelta objects used to define the left and right edges of the time window.
        These can be positive or negative.
        If input value is None, set equal to 0 (datetime.timedelta(days=0)), meaning the returned t_min values
          will be the time the power went out, and the returned t_max values will either be the time the power went out
          (times_relative_to_off_ts_only==True) or the time power was restored (times_relative_to_off_ts_only==False), 
          depending on the value of times_relative_to_off_ts_only.
        The return t_min column is calculated by:
            adding td_for_left to the time the outage began, 
                time_infos_df['t_min'] = pd.to_datetime(time_infos_df['DT_OFF_TS_FULL'])+td_for_left
            Thus, for t_min to be before the outage, td_for_left must be negative.
        The return t_max column is calculated by:
            If times_relative_to_off_ts_only==True:  adding td_for_right to the time the outage began (DT_OFF_TS_FULL)
            If times_relative_to_off_ts_only==False: adding td_for_right to the time the outage ended (DT_ON_TS)
        RESTRICTIONS:    
            When times_relative_to_off_ts_only==True, td_for_left must be less than td_for_right, so that t_min is less
              than t_max.
            However, when times_relative_to_off_ts_only==False, this is not strictly true (although, in most cases, should be true)
            At the end of the day, all that really matters is that the application of td_for_left and td_for_right to
              the off_ts/on_ts columns results in time_infos_df['t_min'] being less than time_infos_df['t_max']
            ----------
            When can td_for_left be greater than td_for_right?
            -----
                Take, for example, an outage that lasts an entire day.  
                  Assume the outage begins off_ts=2022-10-05 12:00:00 and ends on_ts=2022-10-06 12:00:00.
                  Further assume td_for_left = datetime.timedelta(hours=1) and td_for_right = datetime.timedelta(hours=-1), so
                    clearly td_for_left > td_for_right.
                  ==> left  = off_ts+td_for_left = 2022-10-05 13:00:00
                      right = on_ts+td_for_right = 2022-10-06 11:00:00
                  Thus, left<right, and the window makes sense!
    """
    #-------------------------
    if td_for_left is None:
        td_for_left = datetime.timedelta(days=0)
    if td_for_right is None:
        td_for_right = datetime.timedelta(days=0)
    assert(isinstance(td_for_left, datetime.timedelta))
    assert(isinstance(td_for_right, datetime.timedelta))
    #-------------------------
    dovs_outgs = DOVSOutages(                 
        df_construct_type=DFConstructType.kRunSqlQuery, 
        contstruct_df_args=None, 
        init_df_in_constructor=True, 
        build_sql_function=DOVSOutages_SQL.build_sql_outage, 
        build_sql_function_kwargs=dict(
            outg_rec_nbs=rcpo_df.index.get_level_values(outg_rec_nb_idx_lvl).tolist(), 
            from_table_alias='DOV', 
            datetime_col='DT_OFF_TS_FULL', 
            cols_of_interest=[
                'OUTG_REC_NB', 
                dict(field_desc=f"DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)", 
                     alias='DT_OFF_TS_FULL', table_alias_prefix=None), 
                'DT_ON_TS'
            ], 
            field_to_split='outg_rec_nbs'
        )
    )
    #-------------------------
    time_infos_df = dovs_outgs.df
    time_infos_df = Utilities_df.convert_col_type(df=time_infos_df, column='OUTG_REC_NB', to_type=str)
    time_infos_df=time_infos_df.set_index('OUTG_REC_NB')
    #-------------------------
    time_infos_df['t_min'] = pd.to_datetime(time_infos_df['DT_OFF_TS_FULL'])+td_for_left
    if times_relative_to_off_ts_only:
        assert(td_for_left<=td_for_right)
        time_infos_df['t_max'] = pd.to_datetime(time_infos_df['DT_OFF_TS_FULL'])+td_for_right
    else:
        time_infos_df['t_max'] = pd.to_datetime(time_infos_df['DT_ON_TS'])+td_for_right
        assert(all(time_infos_df['t_min'] <= time_infos_df['t_max']))
    #-------------------------
    time_infos_df = time_infos_df.drop(columns=['DT_OFF_TS_FULL', 'DT_ON_TS'])
    #-------------------------
    # Remove any duplicates from time_infos_df
    if time_infos_df.index.nlevels==1:
        tmp_col = Utilities.generate_random_string()
        time_infos_df[tmp_col] = time_infos_df.index
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=[tmp_col])
    else:
        tmp_cols = [Utilities.generate_random_string() for _ in range(time_infos_df.index.nlevels)]
        for i_col, tmp_col in enumerate(tmp_cols):
            time_infos_df[tmp_col] = time_infos_df.index.get_level_values(i_col)
        time_infos_df = time_infos_df.drop_duplicates()
        time_infos_df = time_infos_df.drop(columns=tmp_cols)
    #-------------------------
    return time_infos_df

In [ ]:
# Moved to OutageMdlrPrep
def build_reason_counts_per_outage_from_csvs_NEW_v0(    
    files_dir, 
    file_path_glob, 
    file_path_regex, 
    patterns_to_replace, 
    mp_df, 
    min_outg_td_window=datetime.timedelta(days=1),
    max_outg_td_window=datetime.timedelta(days=30),
    build_ede_typeid_to_reason_df=False, 
    batch_size=50, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    assert_all_cols_equal=True, 
    include_normalize_by_nSNs=True, 
    inclue_zero_counts=True, 
    return_multiindex_outg_reason=False, 
    return_normalized_separately=False, 
    verbose=True, 
    n_update=1, 
    grp_by_cols='outg_rec_nb', 
    outg_rec_nb_col='outg_rec_nb',
    trsf_pole_nb_col = 'trsf_pole_nb', 
    addtnl_dropna_subset_cols=None, 
    is_no_outage=False, 
    prem_nb_col='aep_premise_nb', 
    serial_number_col='serialnumber', 
    include_prem_nbs=False, 
    trust_sql_grouping=True, 
    mp_df_cols = dict(
        serial_number_col='mfr_devc_ser_nbr', 
        prem_nb_col='prem_nb', 
        outg_rec_nb_col='OUTG_REC_NB'
    )
):
    r"""
    Note: The larger the batch_size, the more memory that will be consumed during building

    Any rows with NaNs in outg_rec_nb_col+addtnl_dropna_subset_cols will be dropped

    #NOTE: Currently, only set up for the case return_normalized_separately==False
    """
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # TODO Allow method for reading DOVS from CSV (instead of only from SQL query)
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #-------------------------
    assert(not return_normalized_separately)
    #-------------------------
    assert(Utilities.is_object_one_of_types(grp_by_cols, [str, list, tuple]))
    if isinstance(grp_by_cols, str):
        grp_by_cols = [grp_by_cols]        
    #-------------------------
    # If including prem numbers, also include n_prem numbers
    include_nprem_nbs = include_prem_nbs
    #-------------------------
    # normalize_by_nSNs_included needed when return_multiindex_outg_reason is True
    if include_normalize_by_nSNs and not return_normalized_separately:
        normalize_by_nSNs_included=True
    else:
        normalize_by_nSNs_included=False

    are_dfs_wide_form = not return_multiindex_outg_reason
    is_norm=False #TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #-------------------------
    paths = Utilities.find_all_paths(
        base_dir=files_dir, 
        glob_pattern=file_path_glob, 
        regex_pattern=file_path_regex
    )
    if len(paths)==0:
        print(f'No paths found in files_dir = {files_dir}')
        return None
    paths=natsorted(paths)
    #-------------------------
    rcpo_full = pd.DataFrame()
    #-------------------------
    batch_idxs = Utilities.get_batch_idx_pairs(len(paths), batch_size)
    n_batches = len(batch_idxs)    
    if verbose:
        print(f'n_paths = {len(paths)}')
        print(f'batch_size = {batch_size}')
        print(f'n_batches = {n_batches}')    
    #-------------------------
    for i, batch_i in enumerate(batch_idxs):
        start = time.time()
        if verbose and (i+1)%n_update==0:
            print(f'{i+1}/{n_batches}')
        i_beg = batch_i[0]
        i_end = batch_i[1]
        #-----
        # NOTE: make_all_columns_lowercase=True because...
        #   EMR would return lowercase outg_rec_nb or outg_rec_nb_gpd_for_sql
        #   Athena maintains the original case, and does not conver to lower case,
        #     so it returns OUTG_REC_NB or OUTG_REC_NB_GPD_FOR_SQL
        end_events_df_i = GenAn.read_df_from_csv_batch(
            paths=paths[i_beg:i_end], 
            cols_and_types_to_convert_dict=cols_and_types_to_convert_dict, 
            to_numeric_errors=to_numeric_errors, 
            make_all_columns_lowercase=True, 
            assert_all_cols_equal=assert_all_cols_equal
        )
        print(f'0: {time.time()-start}')
        #-------------------------
        if end_events_df_i.shape[0]==0:
            continue
        #-------------------------
        start = time.time()
        for grp_by_col in grp_by_cols:
            if f'{grp_by_col}_gpd_for_sql' in end_events_df_i.columns:
                if grp_by_col in end_events_df_i.columns:
                    if trust_sql_grouping:
                        end_events_df_i = end_events_df_i.drop(columns=[grp_by_col])
                    else:
                        end_events_df_i = end_events_df_i.drop(columns=[f'{grp_by_col}_gpd_for_sql'])
                end_events_df_i = end_events_df_i.rename(columns={f'{grp_by_col}_gpd_for_sql':grp_by_col})
#             assert(grp_by_col in end_events_df_i.columns)
        #-----
        if not is_no_outage:
            if f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df_i.columns:
                if outg_rec_nb_col in end_events_df_i.columns:
                    if trust_sql_grouping:
                        end_events_df_i = end_events_df_i.drop(columns=[outg_rec_nb_col])
                    else:
                        end_events_df_i = end_events_df_i.drop(columns=[f'{outg_rec_nb_col}_gpd_for_sql'])
                end_events_df_i = end_events_df_i.rename(columns={f'{outg_rec_nb_col}_gpd_for_sql':outg_rec_nb_col})
            assert(outg_rec_nb_col in end_events_df_i.columns)
        #-------------------------
        merge_on_mp = [mp_df_cols['serial_number_col'], mp_df_cols['prem_nb_col']]
        merge_on_ede = [serial_number_col, prem_nb_col]
        if not is_no_outage:
            merge_on_mp.append(mp_df_cols['outg_rec_nb_col'])
            merge_on_ede.append(outg_rec_nb_col)
        if trsf_pole_nb_col in end_events_df_i.columns:
            assert(trsf_pole_nb_col in mp_df.columns)
            merge_on_mp.append(trsf_pole_nb_col)
            merge_on_ede.append(trsf_pole_nb_col)
        # Below ensures there is only one entry per 'meter' (meter here is defined by a unique grouping of merge_on_mp)
        assert(not any(mp_df.groupby(merge_on_mp).size()>1))
        #-----
        # FROM WHAT I CAN TELL, the meters which are 'missing' from meter premise but are present in end events
        #   are caused by meters which were removed or installed close to (a few days/weeks before) an outage.
        # Therefore, although the meter may not have been present at the time of the outage (and therefore was exluced from
        #   meter premise), it could have registered events leading up to/following the outage.
        # e.g., if a meter was removed in the days before an outage, end events are still found for this meter in the days leading up to the outage
        # e.g., if a meter was installed in the days after an outage, end events are still found for this meter in the days following an outage.
        # How should these be handled?
        # The simplest method, which I will implement for now, is to simply ONLY consider those meters which were present
        #   at the time of the outage.  THEREFORE, the two DFs should be joined with an inner merge!
        end_events_df_i = AMIEndEvents.merge_end_events_df_with_mp(
            end_events_df=end_events_df_i, 
            df_mp=mp_df, 
            merge_on_ede=merge_on_ede, 
            merge_on_mp=merge_on_mp, 
            cols_to_include_mp=None, 
            drop_cols = None, 
            rename_cols=None, 
            how='inner', 
            inplace=True
        )
        for grp_by_col in grp_by_cols:
            assert(grp_by_col in end_events_df_i.columns)
        #-------------------------
        dropna_subset_cols = grp_by_cols
        if addtnl_dropna_subset_cols is not None:
            dropna_subset_cols.extend(addtnl_dropna_subset_cols)
        end_events_df_i = end_events_df_i.dropna(subset=dropna_subset_cols)
        #-------------------------
        end_events_df_i=Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=end_events_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        print(f'1: {time.time()-start}')
        start = time.time()
        #---------------------------------------------------------------------------
        # If min_outg_td_window or max_outg_td_window is not None, enforce time restrictions around outages
        if min_outg_td_window is not None or max_outg_td_window is not None:
            #----------
            if not is_no_outage:
                dovs_outgs = DOVSOutages(                 
                    df_construct_type=DFConstructType.kRunSqlQuery, 
                    contstruct_df_args=None, 
                    init_df_in_constructor=True, 
                    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
                    build_sql_function_kwargs=dict(
                        outg_rec_nbs=end_events_df_i[outg_rec_nb_col].unique().tolist(), 
                        from_table_alias='DOV', 
                        datetime_col='DT_OFF_TS_FULL', 
                        cols_of_interest=[
                            'OUTG_REC_NB', 
                            dict(field_desc=f"DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)", 
                                 alias='DT_OFF_TS_FULL', table_alias_prefix=None)
                        ], 
                        field_to_split='outg_rec_nbs'
                    ),
                )
                outg_dt_off_df = dovs_outgs.df
                outg_dt_off_df = Utilities_df.convert_col_type(df=outg_dt_off_df, column='OUTG_REC_NB', to_type=str)
                outg_dt_off_df=outg_dt_off_df.set_index('OUTG_REC_NB')
                outg_dt_off_series = outg_dt_off_df['DT_OFF_TS_FULL']

                end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
                    end_events_df=end_events_df_i, 
                    outg_times_series=outg_dt_off_series, 
                    min_timedelta=min_outg_td_window, 
                    max_timedelta=max_outg_td_window, 
                    outg_rec_nb_col = outg_rec_nb_col, 
                    datetime_col='valuesinterval_local', 
                    assert_one_time_per_group=True
                )
            else:
                no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
                    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
                    output_prem_nbs_col='prem_nbs', 
                    output_t_min_col='t_min', 
                    output_t_max_col='DT_OFF_TS_FULL', 
                    make_addtnl_groupby_idx=True, 
                    include_summary_paths=False
                )
                assert(no_outg_time_infos_df.index.name==trsf_pole_nb_col)
                assert(end_events_df_i[trsf_pole_nb_col].dtype==no_outg_time_infos_df.index.dtype)
                no_outg_time_infos_series = no_outg_time_infos_df['DT_OFF_TS_FULL']    

                end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
                    end_events_df=end_events_df_i, 
                    outg_times_series=no_outg_time_infos_series, 
                    min_timedelta=min_outg_td_window, 
                    max_timedelta=max_outg_td_window, 
                    outg_rec_nb_col = trsf_pole_nb_col, 
                    datetime_col='valuesinterval_local', 
                    assert_one_time_per_group=False
                )
        print(f'2: {time.time()-start}')
        #---------------------------------------------------------------------------
        # After enforcing events within specific time frame, it is possible end_events_df_i is empty.
        # If so, continue
        if end_events_df_i.shape[0]==0:
            continue
        #-------------------------
#         end_events_df_i = AMIEndEvents.extract_reboot_counts_from_reasons_in_df(end_events_df_i)
#         end_events_df_i = AMIEndEvents.extract_fail_reasons_from_reasons_in_df(end_events_df_i)
        #-------------------------
        start = time.time()
        end_events_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(
            df=end_events_df_i, 
#             patterns_to_replace=patterns_to_replace
        )
        print(f'3: {time.time()-start}')
        #-------------------------
        start = time.time()
        if build_ede_typeid_to_reason_df:
            ede_typeid_to_reason_df_i = AMIEndEvents.build_ede_typeid_to_reason_df(
                end_events_df=end_events_df_i, 
                reason_col='reason', 
                ede_typeid_col='enddeviceeventtypeid'
            )
            if i==0:
                ede_typeid_to_reason_df = ede_typeid_to_reason_df_i.copy()
            else:
                ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
                    ede_typeid_to_reason_df1=ede_typeid_to_reason_df, 
                    ede_typeid_to_reason_df2=ede_typeid_to_reason_df_i,
                    sort=True
                )
        print(f'4: {time.time()-start}')
        #-------------------------
#         ordinal_encoder = OrdinalEncoder()
#         end_events_df_i['reason_enc'] = ordinal_encoder.fit_transform(end_events_df_i[['reason']])
        #-------------------------
        start = time.time()
        rcpo_i = AMIEndEvents.get_reason_counts_per_group(
            end_events_df = end_events_df_i, 
            group_cols=grp_by_cols, 
            group_freq=None, 
            serial_number_col='serialnumber', 
            reason_col='reason', 
            include_normalize_by_nSNs=include_normalize_by_nSNs, 
            inclue_zero_counts=inclue_zero_counts,
            possible_reasons=None, 
            include_nSNs=True, 
            include_SNs=True, 
            prem_nb_col=prem_nb_col, 
            include_nprem_nbs=include_nprem_nbs,
            include_prem_nbs=include_prem_nbs,   
            return_form = dict(return_multiindex_outg_reason = return_multiindex_outg_reason, 
                               return_normalized_separately  = return_normalized_separately)
        )
        print(f'5: {time.time()-start}')
        #-------------------------
        if rcpo_i.shape[0]==0:
            continue
        #-------------------------
        start = time.time()
        # Include or below in case i=0 rcpo_i comes back empty, causing continue to be called above
        if i==0 or rcpo_full.shape[0]==0:
            rcpo_full = rcpo_i.copy()
        else:
            list_cols=['_SNs']
            list_counts_cols=['_nSNs']
            w_col = '_nSNs'
            if include_prem_nbs:
                list_cols.append('_prem_nbs')
                list_counts_cols.append('_nprem_nbs')
            #-----
            rcpo_full = AMIEndEvents.combine_two_reason_counts_per_outage_dfs(
                rcpo_df_1=rcpo_full, 
                rcpo_df_2=rcpo_i, 
                are_dfs_wide_form=are_dfs_wide_form, 
                normalize_by_nSNs_included=normalize_by_nSNs_included, 
                is_norm=is_norm, 
                list_cols=list_cols, 
                list_counts_cols=list_counts_cols,
                w_col = w_col, 
                level_0_raw_col = 'counts', 
                level_0_nrm_col = 'counts_norm', 
                convert_rcpo_wide_to_long_col_args=None
            )
        print(f'6: {time.time()-start}')
    if not build_ede_typeid_to_reason_df:
        return rcpo_full
    else:
        return rcpo_full, ede_typeid_to_reason_df

In [ ]:
# Moved to OutageMdlrPrep
def check_end_events_merge_with_mp(
    ede_file_paths,
    mp_df, 
    threshold_pct=1.0, 
    outg_rec_nb_col='outg_rec_nb',
    trsf_pole_nb_col = 'trsf_pole_nb', 
    prem_nb_col='aep_premise_nb', 
    serial_number_col='serialnumber',
    mp_df_cols = dict(
        serial_number_col='mfr_devc_ser_nbr', 
        prem_nb_col='prem_nb', 
        trsf_pole_nb_col = 'trsf_pole_nb', 
        outg_rec_nb_col='OUTG_REC_NB'
    ), 
    is_no_outage=False, 
    assert_all_cols_equal=True
):
    r"""
    When building the RCPX dfs, oftentimes the end events data need to be joined with meter premise data (e.g., when one
      wants to group by transformer, this information typically needs to be brought in from MP).
    The meter premise data are supplied by the user.  Mistakes can be made, so this function serves to check whether the 
      alignment between end events and meter premise is as expected.
      
    NOTE: This also runs some other checks on the end events collection as a whole (e.g., if assert_all_cols_equal, it
            checks the assertion).
          Thus, some of the operations can be replaced from the for loop in build_reason_counts_per_outage_from_csv
            (This will help performance, but I don't think gains will be significant)
            
    In order to reduce some of the load on build_reason_counts_per_outage_from_csv, this function returns merge_on_ede and merge_on_mp
      
    End events and meter premise are merged on serial number and premise number.
      If data are for outages, also merged on outg_rec_nb.
      If trsf_pole_nb is present, also merged on it.
      
    NOTE:  I have found there are meters which are 'missing' from meter premise but are present in end events.
           These seem to be caused by meters which were removed or installed close to (a few days/weeks before) an outage.
           Therefore, although the meter may not have been present at the time of the outage (and therefore was exluced from
             meter premise), it could have registered events leading up to/following the outage.
           e.g., if a meter was removed in the days before an outage, end events are still found for this meter in the days leading up to the outage
           e.g., if a meter was installed in the days after an outage, end events are still found for this meter in the days following an outage.
           How should these be handled?
           The simplest method, which I will implement for now, is to simply ONLY consider those meters which were present
             at the time of the outage.  THEREFORE, the two DFs should be joined with an inner merge! 
             
    NOTE: Utilities_df.make_all_column_names_lowercase utilized when reading in end events dfs because...
      EMR would return lowercase outg_rec_nb or outg_rec_nb_gpd_for_sql
      Athena maintains the original case, and does not conver to lower case,
        so it returns OUTG_REC_NB or OUTG_REC_NB_GPD_FOR_SQL
             
    threshold_pct:
        If the percentage of entries present in end events but absent in meter premise exceeds this value, the program will crash.
        This is to safeguard the user from inputting incorrect meter premise data.
    """
    #-------------------------
    merge_on_mp = [mp_df_cols['serial_number_col'], mp_df_cols['prem_nb_col']]
    merge_on_ede = [serial_number_col, prem_nb_col]
    if not is_no_outage:
        merge_on_mp.append(mp_df_cols['outg_rec_nb_col'])
        merge_on_ede.append(outg_rec_nb_col)
    #-------------------------
    # Grab the first end events df and check for trsf_pole_nb
    #  Actually, just need to grab first entry of first df
    end_events_df_0 = GenAn.read_df_from_csv(
        read_path=ede_file_paths[0], 
        cols_and_types_to_convert_dict=None, 
        to_numeric_errors='coerce', 
        drop_na_rows_when_exception=True, 
        drop_unnamed0_col=True, 
        pd_read_csv_kwargs = dict(nrows=1)
    )
    end_events_df_0 = Utilities_df.make_all_column_names_lowercase(end_events_df_0)
    #-----
    if trsf_pole_nb_col in end_events_df_0.columns:
        assert(mp_df_cols['trsf_pole_nb_col'] in mp_df.columns)
        merge_on_mp.append(mp_df_cols['trsf_pole_nb_col'])
        merge_on_ede.append(trsf_pole_nb_col)
    #-------------------------
    # Below ensures there is only one entry per 'meter' (meter here is defined by a unique grouping of merge_on_mp)
    assert(not any(mp_df.groupby(merge_on_mp).size()>1))
    #--------------------------------------------------
    # Build DF with all end events to be checked with mp_df
    # The only columns which matter for the check are those in merge_on_ede, so those are the only kept.
    #   This keeps the overall size of the end_events_df smaller, allowing all to be loaded at once in most cases.
    dfs = []
    for path in ede_file_paths:
        df_i = GenAn.read_df_from_csv(
            read_path=path, 
            cols_and_types_to_convert_dict=None, 
            to_numeric_errors='coerce', 
            drop_na_rows_when_exception=True, 
            drop_unnamed0_col=True
        )
        if df_i.shape[0]==0:
            continue
        #-----
        df_i = Utilities_df.make_all_column_names_lowercase(df_i) 
        #-----
        dfs.append(df_i[merge_on_ede])
    #-------------------------                
    df_cols = Utilities_df.get_shared_columns(dfs, maintain_df0_order=True)
    for i in range(len(dfs)):
        if assert_all_cols_equal:
            # In order to account for case where columns are the same but in different order
            # one must compare the length of dfs[i].columns to that of df_cols (found by utilizing
            # the Utilities_df.get_shared_columns(dfs) functionality)
            assert(dfs[i].shape[1]==len(df_cols))
        dfs[i] = dfs[i][df_cols]
    end_events_df = pd.concat(dfs)
    #-------------------------
    # Alter dtypes in mp_df if needed for proper merging
    end_events_df, mp_df = Utilities_df.make_df_col_dtypes_equal(
        df_1              = end_events_df, 
        col_1             = merge_on_ede, 
        df_2              = mp_df, 
        col_2             = merge_on_mp, 
        allow_reverse_set = False, 
        assert_success    = True
    )
    #-------------------------
    # Now, compare the unique combinations of merge_on_ede in end_events_df to those of merge_on_mp in mp_df
    gps_ede = list(end_events_df.groupby(merge_on_ede).groups.keys())
    gps_mp  = list(mp_df.groupby(merge_on_mp).groups.keys())
    # Finally, find the percent of meters in end events missing from meter premise (where a meter is defined as a unique
    #   combination of merge_on_ede values (e.g., serial number and premise number))
    n_missing = len(set(gps_ede).difference(set(gps_mp)))
    pct_missing = 100*(n_missing/len(gps_ede))
    print(f'% meters in end events missing from mp_df: {pct_missing}')
    if pct_missing>threshold_pct:
        assert(0)
    #-------------------------
    return mp_df, merge_on_ede, merge_on_mp

In [ ]:
# Moved to OutageMdlrPrep
def perform_build_RCPX_from_csvs_prereqs_v0(
    files_dir, 
    file_path_glob, 
    file_path_regex,
    mp_df, 
    ede_mp_mismatch_threshold_pct=1.0, 
    grp_by_cols='outg_rec_nb', 
    outg_rec_nb_col='outg_rec_nb',
    trsf_pole_nb_col = 'trsf_pole_nb', 
    prem_nb_col='aep_premise_nb', 
    serial_number_col='serialnumber',
    mp_df_cols = dict(
        serial_number_col='mfr_devc_ser_nbr', 
        prem_nb_col='prem_nb', 
        trsf_pole_nb_col = 'trsf_pole_nb', 
        outg_rec_nb_col='OUTG_REC_NB'
    ), 
    is_no_outage=False, 
    assert_all_cols_equal=True, 
    trust_sql_grouping=True
):
    r"""
    Prepares for running build_reason_counts_per_outage_from_csvs.
    Many of these items were done for each iteration of the main for loop in build_reason_counts_per_outage_from_csvs, which
      was really unnecessary.  
      This will improve performance (although, likely not much as none of the operations are super heavy) and simplify the code.
    
    1. Adjust grp_by_cols and outg_rec_nb_col to handle cases where _gpd_for_sql appendix was added during data acquisition.
       If there is an instance where, e.g., outg_rec_nb_col and f'{outg_rec_nb_col}_gpd_for_sql' are both present, it settles
         the discrepancy (according to the trust_sql_grouping parameter) and compiles a list of columns which will need 
         to be dropped.
    2. Determine merge_on_ede and merge_on_mp columns (done within check_end_events_merge_with_mp).
    3. Checks that the user supplied mp_df aligns well with the end events data.
    4. If assert_all_cols_equal==True, enforce the assertion.
    """
    #-------------------------
    assert(Utilities.is_object_one_of_types(grp_by_cols, [str, list, tuple]))
    if isinstance(grp_by_cols, str):
        grp_by_cols = [grp_by_cols]    
    #-------------------------
    paths = Utilities.find_all_paths(
        base_dir=files_dir, 
        glob_pattern=file_path_glob, 
        regex_pattern=file_path_regex
    )
    if len(paths)==0:
        print(f'No paths found in files_dir = {files_dir}')
        return None
    paths=natsorted(paths)
    #-------------------------
    #--------------------------------------------------
    # Grab first row from each CSV file to be used to check columns
    dfs = []
    for path in paths:
        df_i = GenAn.read_df_from_csv(
            read_path=path, 
            cols_and_types_to_convert_dict=None, 
            to_numeric_errors='coerce', 
            drop_na_rows_when_exception=True, 
            drop_unnamed0_col=True, 
            pd_read_csv_kwargs = dict(nrows=1)
        )
        if df_i.shape[0]==0:
            continue
        #-----
        # NOTE: make_all_columns_lowercase because...
        #   EMR would return lowercase outg_rec_nb or outg_rec_nb_gpd_for_sql
        #   Athena maintains the original case, and does not conver to lower case,
        #     so it returns OUTG_REC_NB or OUTG_REC_NB_GPD_FOR_SQL
        df_i = Utilities_df.make_all_column_names_lowercase(df_i) 
        #-----
        dfs.append(df_i)
    #-------------------------
    if len(dfs)==0:
        print(f'No DFs found in files_dir={files_dir}')
        assert(0)
    #-------------------------                
    df_cols = Utilities_df.get_shared_columns(dfs, maintain_df0_order=True)
    for i in range(len(dfs)):
        if assert_all_cols_equal:
            # In order to account for case where columns are the same but in different order
            # one must compare the length of dfs[i].columns to that of df_cols (found by utilizing
            # the Utilities_df.get_shared_columns(dfs) functionality)
            assert(dfs[i].shape[1]==len(df_cols))
        dfs[i] = dfs[i][df_cols]
    end_events_df = pd.concat(dfs)
    #--------------------------------------------------
    #--------------------------------------------------
    # The columns in grp_by_cols may sometimes be appended with _gpd_for_sql (e.g., typically one will see outg_rec_nb_gpd_for_sql 
    #   not outg_rec_nb in the raw CSV files).
    # The _gpd_for_sql was appended during data acquisition.
    # However, the user typically does not remember this, and will usually input, e.g., outg_rec_nb_col='outg_rec_nb'
    #   Such a scenario will obviously lead to an error, as 'outg_rec_nb' is not found in the columns
    # The below methods serve to remedy that issue.
    # NOTE: In the case that column_i and column_i_gpd_for_sql are for whatever reason BOTH found in the DF, the parameter
    #       trust_sql_grouping directs the code on which to use (the other will be dropped)
    cols_to_drop = []
    #-------------------------
    grp_by_cols_final = []
    for grp_by_col in grp_by_cols:
        if f'{grp_by_col}_gpd_for_sql' in end_events_df.columns:
            if grp_by_col in end_events_df.columns:
                # Both f'{grp_by_col}_gpd_for_sql' and grp_by_col.
                # One must be kept (by inserting into grp_by_cols_final), and one dropped (by inserting into cols_to_drop)
                if trust_sql_grouping:
                    grp_by_cols_final.append(f'{grp_by_col}_gpd_for_sql')
                    cols_to_drop.append(grp_by_col)
                else:
                    grp_by_cols_final.append(grp_by_col)
                    cols_to_drop.append(f'{grp_by_col}_gpd_for_sql')
            else:
                # Only f'{grp_by_col}_gpd_for_sql', not grp_by_col, so no need to drop anything
                grp_by_cols_final.append(f'{grp_by_col}_gpd_for_sql')
        else:
            grp_by_cols_final.append(grp_by_col)
    grp_by_cols = grp_by_cols_final
    #-------------------------
    if not is_no_outage:
        if f'{outg_rec_nb_col}_gpd_for_sql' in end_events_df.columns:
            if outg_rec_nb_col in end_events_df.columns:
                # Both f'{outg_rec_nb_col}_gpd_for_sql' and outg_rec_nb_col.
                # One must be kept (by setting equal to outg_rec_nb_col), and one dropped (by inserting into cols_to_drop)
                if trust_sql_grouping:
                    cols_to_drop.append(outg_rec_nb_col)
                    outg_rec_nb_col = f'{outg_rec_nb_col}_gpd_for_sql'
                else:
                    cols_to_drop.append(f'{outg_rec_nb_col}_gpd_for_sql')
                    outg_rec_nb_col = outg_rec_nb_col
            else:
                # Only f'{outg_rec_nb_col}_gpd_for_sql', not outg_rec_nb_col, so no need to drop anything
                outg_rec_nb_col = f'{outg_rec_nb_col}_gpd_for_sql'
        assert(outg_rec_nb_col in end_events_df.columns)
    #-------------------------
    if f'{trsf_pole_nb_col}_gpd_for_sql' in end_events_df.columns:
        if trsf_pole_nb_col in end_events_df.columns:
            # Both f'{trsf_pole_nb_col}_gpd_for_sql' and trsf_pole_nb_col.
            # One must be kept (by setting equal to trsf_pole_nb_col), and one dropped (by inserting into cols_to_drop)
            if trust_sql_grouping:
                cols_to_drop.append(trsf_pole_nb_col)
                trsf_pole_nb_col = f'{trsf_pole_nb_col}_gpd_for_sql'
            else:
                cols_to_drop.append(f'{trsf_pole_nb_col}_gpd_for_sql')
                trsf_pole_nb_col = trsf_pole_nb_col
        else:
            # Only f'{trsf_pole_nb_col}_gpd_for_sql', not trsf_pole_nb_col, so no need to drop anything
            trsf_pole_nb_col = f'{trsf_pole_nb_col}_gpd_for_sql'        
    #-------------------------
    # NOTE: Many times outg_rec_nb_col is in grp_by_cols, hence the need for the set operation (to eliminate duplicates!)
    cols_to_drop = list(set(cols_to_drop))
    #--------------------------------------------------
    #--------------------------------------------------
    mp_df, merge_on_ede, merge_on_mp = check_end_events_merge_with_mp(
        ede_file_paths=paths,
        mp_df=mp_df, 
        threshold_pct=ede_mp_mismatch_threshold_pct, 
        outg_rec_nb_col=outg_rec_nb_col,
        trsf_pole_nb_col=trsf_pole_nb_col, 
        prem_nb_col=prem_nb_col, 
        serial_number_col=serial_number_col,
        mp_df_cols = mp_df_cols, 
        is_no_outage=is_no_outage, 
        assert_all_cols_equal=assert_all_cols_equal
    )
    #--------------------------------------------------
    #--------------------------------------------------
    return_dict = dict(
        paths=paths, 
        grp_by_cols=grp_by_cols, 
        outg_rec_nb_col=outg_rec_nb_col, 
        trsf_pole_nb_col=trsf_pole_nb_col, 
        cols_to_drop=cols_to_drop, 
        mp_df=mp_df, 
        merge_on_ede=merge_on_ede, 
        merge_on_mp=merge_on_mp
    )
    return return_dict

In [ ]:
# Moved to OutageMdlrPrep
def find_all_cols_with_gpd_for_sql_appendix(df):
    r"""
    The _gpd_for_sql appendix is added in the end events data acquisition stage, but is typically no longer needed after.
    NOTE: Method is case insensitive!! (i.e., col_gpd_for_sql --> col, col_GPD_FOR_SQL --> col, etc.)
    Returns a dict with key values equal to the found columns containing _gpd_for_sql and values w
    """
    #-------------------------
    found_cols_dict = {}
    for col in df.columns.tolist():
        if col.lower().endswith('_gpd_for_sql'):
            assert(col not in found_cols_dict.keys())
            found_cols_dict[col] = col[:-12]
    return found_cols_dict

In [ ]:
# Moved to OutageMdlrPrep
def identify_ede_cols_of_interest_to_update_andor_drop(
    end_events_df,  
    grp_by_cols        = 'outg_rec_nb', 
    outg_rec_nb_col    = 'outg_rec_nb',
    trsf_pole_nb_col   = 'trsf_pole_nb', 
    prem_nb_col        = 'aep_premise_nb', 
    serial_number_col  = 'serialnumber',
    trust_sql_grouping = True
):
    r"""
    Find any entries in ede_cols_of_interest (see below) which contain _gpd_for_sql appendix and update accordingly.
      The columns in end events may sometimes be appended with _gpd_for_sql (e.g., typically one will see outg_rec_nb_gpd_for_sql 
        not outg_rec_nb in the raw CSV files).
      The _gpd_for_sql was appended during data acquisition.
      However, the user typically does not remember this, and will usually input, e.g., outg_rec_nb_col='outg_rec_nb'
        Such a scenario will obviously lead to an error, as 'outg_rec_nb' is not found in the columns
      The below methods serve to remedy that issue.
      
    ede_cols_of_interest consist of grp_by_cols+[outg_rec_nb_col, trsf_pole_nb_col, prem_nb_col, serial_number_col]
      
    NOTE: In the case that column_i and column_i_gpd_for_sql are for whatever reason BOTH found in the DF, the parameter
        trust_sql_grouping directs the code on which to use (the other will be dropped)
    """
    #-------------------------
    assert(Utilities.is_object_one_of_types(grp_by_cols, [str, list, tuple]))
    if isinstance(grp_by_cols, str):
        grp_by_cols = [grp_by_cols]    
    #-------------------------
    # Below called ede_gpd_coi_dict because the columns of interest are grouped by their input parameter in build_reason_counts_per_outage_from_csvs.
    #   e.g., key value grp_by_cols should contain a list of columns (strings)
    #         key value outg_rec_nb_col contains a single string for the outg_rec_nb_col
    # The values, coi, can stand for column of interest (e.g., outg_rec_nb_col) or columns of interest (e.g., grp_by_cols)
    ede_gpd_coi_dict = dict(
        grp_by_cols=grp_by_cols, 
        outg_rec_nb_col=outg_rec_nb_col, 
        trsf_pole_nb_col=trsf_pole_nb_col, 
        prem_nb_col=prem_nb_col, 
        serial_number_col=serial_number_col
    )
    #-----
    # Below, ede_cols_of_interest is essentially the flattened values from ede_gpd_coi_dict with duplicates removed
    ede_cols_of_interest = []
    for ede_coi_grp, coi in ede_gpd_coi_dict.items():
        assert(Utilities.is_object_one_of_types(coi, [str,list,tuple]))
        if isinstance(coi, str):
            ede_cols_of_interest.append(coi)
        else:
            ede_cols_of_interest.extend(coi)
    # NOTE: grp_by_cols can contain the others (e.g., outg_rec_nb_col), so set operation needed to removed duplicates
    ede_cols_of_interest = list(set(ede_cols_of_interest))
    #--------------------------------------------------
    # Below, found_cols_w_gpd_for_sql_appendix has keys equal to any columns found containing _gpd_for_sql appendix and
    #   value equal to the column without the appendix.
    # found_cols_w_gpd_for_sql_appendix_inv is the inverse (i.e., keys and values switched)
    found_cols_w_gpd_for_sql_appendix = find_all_cols_with_gpd_for_sql_appendix(end_events_df)
    found_cols_w_gpd_for_sql_appendix_inv = Utilities.invert_dict(found_cols_w_gpd_for_sql_appendix)
    #--------------------------------------------------
    # NOTE: No harm below if ede_coi not actually found in end_events_df (e.g., when running over baseline data, outg_rec_nb_col 
    #       will not be found), as updates are only made to those whose values change (and can only change if found)
    ede_cols_of_interest_updates = dict()
    cols_to_drop = []
    #-----
    for ede_coi in ede_cols_of_interest:
        assert(ede_coi not in ede_cols_of_interest_updates)
        #-----
        if ede_coi in found_cols_w_gpd_for_sql_appendix_inv.keys():
            # f'{ede_coi}_gpd_for_sql' found in end_events_df, so an updadte to ede_coi in ede_cols_of_interest 
            #   and/or column drop will be needed
            if ede_coi in end_events_df.columns:
                # Both f'{ede_coi}_gpd_for_sql' and ede_coi were found in end_events_df!
                # One must be kept (by inserting into grp_by_cols_final), and one dropped (by inserting into cols_to_drop)
                if trust_sql_grouping:
                    # Keep/update f'{ede_coi}_gpd_for_sql', and drop ede_coi
                    ede_cols_of_interest_updates[ede_coi] = found_cols_w_gpd_for_sql_appendix_inv[ede_coi]
                    cols_to_drop.append(ede_coi)
                else:
                    # Keep ede_coi, drop f'{ede_coi}_gpd_for_sql'
                    ede_cols_of_interest_updates[ede_coi] = ede_coi
                    cols_to_drop.append(found_cols_w_gpd_for_sql_appendix_inv[ede_coi])
            else:
                # Only f'{ede_coi}_gpd_for_sql' found in end_events_df, not grp_by_col, so no need to drop anything
                ede_cols_of_interest_updates[ede_coi] = found_cols_w_gpd_for_sql_appendix_inv[ede_coi]
        else:
            ede_cols_of_interest_updates[ede_coi] = ede_coi
    #-----
    # Make sure no repeats in cols_to_drop
    cols_to_drop = list(set(cols_to_drop))
    #--------------------------------------------------
    # Build ede_gpd_coi_dict_updates, which has the same structure as ede_gpd_coi_dict (see above)
    ede_gpd_coi_dict_updates = dict()
    for ede_coi_grp, coi in ede_gpd_coi_dict.items():
        assert(ede_coi_grp not in ede_gpd_coi_dict_updates.keys())
        assert(Utilities.is_object_one_of_types(coi, [str,list,tuple]))
        if isinstance(coi, str):
            assert(coi in ede_cols_of_interest_updates.keys())
            ede_gpd_coi_dict_updates[ede_coi_grp] = ede_cols_of_interest_updates[coi]
        else:
            ede_gpd_coi_dict_updates[ede_coi_grp] = []
            for coi_i in coi:
                assert(coi_i in ede_cols_of_interest_updates.keys())
                ede_gpd_coi_dict_updates[ede_coi_grp].append(ede_cols_of_interest_updates[coi_i])
    assert(len(set(ede_gpd_coi_dict.keys()).symmetric_difference(set(ede_gpd_coi_dict_updates.keys())))==0)
    #-----
    # Sanity check
    assert(len(set(ede_gpd_coi_dict.keys()).symmetric_difference(set(ede_gpd_coi_dict_updates.keys())))==0)
    for ede_coi_grp, coi in ede_gpd_coi_dict.items():
        if Utilities.is_object_one_of_types(coi, [list,tuple]):
            assert(len(coi)==len(ede_gpd_coi_dict_updates[ede_coi_grp]))
    #--------------------------------------------------
    return ede_gpd_coi_dict_updates, cols_to_drop

In [ ]:
# Moved to OutageMdlrPrep
def perform_build_RCPX_from_csvs_prereqs(
    files_dir, 
    file_path_glob, 
    file_path_regex,
    mp_df, 
    ede_mp_mismatch_threshold_pct=1.0, 
    grp_by_cols='outg_rec_nb', 
    outg_rec_nb_col='outg_rec_nb',
    trsf_pole_nb_col = 'trsf_pole_nb', 
    prem_nb_col='aep_premise_nb', 
    serial_number_col='serialnumber',
    mp_df_cols = dict(
        serial_number_col='mfr_devc_ser_nbr', 
        prem_nb_col='prem_nb', 
        trsf_pole_nb_col = 'trsf_pole_nb', 
        outg_rec_nb_col='OUTG_REC_NB'
    ), 
    is_no_outage=False, 
    assert_all_cols_equal=True, 
    trust_sql_grouping=True
):
    r"""
    Prepares for running build_reason_counts_per_outage_from_csvs.
    Many of these items were done for each iteration of the main for loop in build_reason_counts_per_outage_from_csvs, which
      was really unnecessary.  
      This will improve performance (although, likely not much as none of the operations are super heavy) and simplify the code.
    
    1. Adjust grp_by_cols and outg_rec_nb_col to handle cases where _gpd_for_sql appendix was added during data acquisition.
       If there is an instance where, e.g., outg_rec_nb_col and f'{outg_rec_nb_col}_gpd_for_sql' are both present, it settles
         the discrepancy (according to the trust_sql_grouping parameter) and compiles a list of columns which will need 
         to be dropped.
    2. Determine merge_on_ede and merge_on_mp columns (done within check_end_events_merge_with_mp).
    3. Checks that the user supplied mp_df aligns well with the end events data.
    4. If assert_all_cols_equal==True, enforce the assertion.
    """
    #-------------------------
    assert(Utilities.is_object_one_of_types(grp_by_cols, [str, list, tuple]))
    if isinstance(grp_by_cols, str):
        grp_by_cols = [grp_by_cols]    
    #-------------------------
    paths = Utilities.find_all_paths(
        base_dir=files_dir, 
        glob_pattern=file_path_glob, 
        regex_pattern=file_path_regex
    )
    if len(paths)==0:
        print(f'No paths found in files_dir = {files_dir}')
        return None
    paths=natsorted(paths)
    #-------------------------
    #--------------------------------------------------
    # Grab first row from each CSV file to be used to check columns
    dfs = []
    for path in paths:
        df_i = GenAn.read_df_from_csv(
            read_path=path, 
            cols_and_types_to_convert_dict=None, 
            to_numeric_errors='coerce', 
            drop_na_rows_when_exception=True, 
            drop_unnamed0_col=True, 
            pd_read_csv_kwargs = dict(nrows=1)
        )
        if df_i.shape[0]==0:
            continue
        #-----
        # NOTE: make_all_columns_lowercase because...
        #   EMR would return lowercase outg_rec_nb or outg_rec_nb_gpd_for_sql
        #   Athena maintains the original case, and does not conver to lower case,
        #     so it returns OUTG_REC_NB or OUTG_REC_NB_GPD_FOR_SQL
        df_i = Utilities_df.make_all_column_names_lowercase(df_i) 
        #-----
        dfs.append(df_i)
    #-------------------------
    if len(dfs)==0:
        print(f'No DFs found in files_dir={files_dir}')
        assert(0)
    #-------------------------                
    df_cols = Utilities_df.get_shared_columns(dfs, maintain_df0_order=True)
    for i in range(len(dfs)):
        if assert_all_cols_equal:
            # In order to account for case where columns are the same but in different order
            # one must compare the length of dfs[i].columns to that of df_cols (found by utilizing
            # the Utilities_df.get_shared_columns(dfs) functionality)
            assert(dfs[i].shape[1]==len(df_cols))
        dfs[i] = dfs[i][df_cols]
    end_events_df = pd.concat(dfs)
    #--------------------------------------------------
    #--------------------------------------------------
    ede_cols_of_interest_updates, cols_to_drop = identify_ede_cols_of_interest_to_update_andor_drop(
        end_events_df      = end_events_df,  
        grp_by_cols        = grp_by_cols, 
        outg_rec_nb_col    = outg_rec_nb_col,
        trsf_pole_nb_col   = trsf_pole_nb_col, 
        prem_nb_col        = prem_nb_col, 
        serial_number_col  = serial_number_col,
        trust_sql_grouping = trust_sql_grouping
    )
    grp_by_cols       = ede_cols_of_interest_updates['grp_by_cols']
    outg_rec_nb_col   = ede_cols_of_interest_updates['outg_rec_nb_col']
    trsf_pole_nb_col  = ede_cols_of_interest_updates['trsf_pole_nb_col']
    prem_nb_col       = ede_cols_of_interest_updates['prem_nb_col']
    serial_number_col = ede_cols_of_interest_updates['serial_number_col']
    #--------------------------------------------------
    #--------------------------------------------------
    mp_df, merge_on_ede, merge_on_mp = check_end_events_merge_with_mp(
        ede_file_paths        = paths,
        mp_df                 = mp_df, 
        threshold_pct         = ede_mp_mismatch_threshold_pct, 
        outg_rec_nb_col       = outg_rec_nb_col,
        trsf_pole_nb_col      = trsf_pole_nb_col, 
        prem_nb_col           = prem_nb_col, 
        serial_number_col     = serial_number_col,
        mp_df_cols            = mp_df_cols, 
        is_no_outage          = is_no_outage, 
        assert_all_cols_equal = assert_all_cols_equal
    )
    #--------------------------------------------------
    #--------------------------------------------------
    return_dict = dict(
        paths=paths, 
        grp_by_cols=grp_by_cols, 
        outg_rec_nb_col=outg_rec_nb_col, 
        trsf_pole_nb_col=trsf_pole_nb_col, 
        cols_to_drop=cols_to_drop, 
        mp_df=mp_df, 
        merge_on_ede=merge_on_ede, 
        merge_on_mp=merge_on_mp
    )
    return return_dict

In [ ]:
# Moved to OutageMdlrPrep
def drop_gpd_for_sql_appendix_from_all_cols(df):
    r"""
    The _gpd_for_sql appendix is added in the end events data acquisition stage, but is typically no longer needed after.
    This function removes the appendix.
    
    NOTE: Method is case insensitive!! (i.e., col_gpd_for_sql --> col, col_GPD_FOR_SQL --> col, etc.)
    """
    #-------------------------
    for col in df.columns.tolist():
        if col.lower().endswith('_gpd_for_sql'):
            df = df.rename(columns={col:col[:-12]})
    return df


# Moved to OutageMdlrPrep
def drop_gpd_for_sql_appendix_from_all_index_names(df):
    r"""
    The _gpd_for_sql appendix is added in the end events data acquisition stage, but is typically no longer needed after.
    This function removes the appendix.
    
    NOTE: Method is case insensitive!! (i.e., col_gpd_for_sql --> col, col_GPD_FOR_SQL --> col, etc.)
    """
    #-------------------------
    idx_names_new = []
    for name in df.index.names:
        if name.lower().endswith('_gpd_for_sql'):
            idx_names_new.append(name[:-12])
        else:
            idx_names_new.append(name)
    #-----
    assert(len(df.index.names)==len(idx_names_new))
    df.index.names = idx_names_new
    return df

In [ ]:
# Moved to OutageMdlrPrep
def build_reason_counts_per_outage_from_csvs_NEW(    
    files_dir, 
    file_path_glob, 
    file_path_regex, 
    patterns_to_replace, 
    mp_df, 
    min_outg_td_window=datetime.timedelta(days=1),
    max_outg_td_window=datetime.timedelta(days=30),
    build_ede_typeid_to_reason_df=False, 
    batch_size=50, 
    cols_and_types_to_convert_dict=None, 
    to_numeric_errors='coerce', 
    assert_all_cols_equal=True, 
    include_normalize_by_nSNs=True, 
    inclue_zero_counts=True, 
    return_multiindex_outg_reason=False, 
    return_normalized_separately=False, 
    verbose=True, 
    n_update=1, 
    grp_by_cols='outg_rec_nb', 
    outg_rec_nb_col='outg_rec_nb',
    trsf_pole_nb_col = 'trsf_pole_nb', 
    addtnl_dropna_subset_cols=None, 
    is_no_outage=False, 
    prem_nb_col='aep_premise_nb', 
    serial_number_col='serialnumber', 
    include_prem_nbs=False, 
    trust_sql_grouping=True, 
    mp_df_cols = dict(
        serial_number_col='mfr_devc_ser_nbr', 
        prem_nb_col='prem_nb', 
        trsf_pole_nb_col = 'trsf_pole_nb', 
        outg_rec_nb_col='OUTG_REC_NB'
    )
):
    r"""
    Note: The larger the batch_size, the more memory that will be consumed during building

    Any rows with NaNs in outg_rec_nb_col+addtnl_dropna_subset_cols will be dropped

    #NOTE: Currently, only set up for the case return_normalized_separately==False
    """
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # TODO Allow method for reading DOVS from CSV (instead of only from SQL query)
    #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #-------------------------
    assert(not return_normalized_separately)
    #-------------------------
    assert(Utilities.is_object_one_of_types(grp_by_cols, [str, list, tuple]))
    if isinstance(grp_by_cols, str):
        grp_by_cols = [grp_by_cols]        
    #-------------------------
    # If including prem numbers, also include n_prem numbers
    include_nprem_nbs = include_prem_nbs
    #-------------------------
    # normalize_by_nSNs_included needed when return_multiindex_outg_reason is True
    if include_normalize_by_nSNs and not return_normalized_separately:
        normalize_by_nSNs_included=True
    else:
        normalize_by_nSNs_included=False

    are_dfs_wide_form = not return_multiindex_outg_reason
    is_norm=False #TODO!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    #-------------------------
    prereq_dict = perform_build_RCPX_from_csvs_prereqs(
        files_dir=files_dir, 
        file_path_glob=file_path_glob, 
        file_path_regex=file_path_regex,
        mp_df=mp_df, 
        ede_mp_mismatch_threshold_pct=1.0, 
        grp_by_cols=grp_by_cols, 
        outg_rec_nb_col=outg_rec_nb_col,
        trsf_pole_nb_col=trsf_pole_nb_col, 
        prem_nb_col=prem_nb_col, 
        serial_number_col=serial_number_col,
        mp_df_cols=mp_df_cols, 
        is_no_outage=is_no_outage, 
        assert_all_cols_equal=assert_all_cols_equal
    )
    paths            = prereq_dict['paths']
    grp_by_cols      = prereq_dict['grp_by_cols']
    outg_rec_nb_col  = prereq_dict['outg_rec_nb_col']
    trsf_pole_nb_col = prereq_dict['trsf_pole_nb_col']
    cols_to_drop     = prereq_dict['cols_to_drop']
    mp_df            = prereq_dict['mp_df']
    merge_on_ede     = prereq_dict['merge_on_ede']
    merge_on_mp      = prereq_dict['merge_on_mp']
    #-------------------------
    rcpo_full = pd.DataFrame()
    #-------------------------
    batch_idxs = Utilities.get_batch_idx_pairs(len(paths), batch_size)
    n_batches = len(batch_idxs)    
    if verbose:
        print(f'n_paths = {len(paths)}')
        print(f'batch_size = {batch_size}')
        print(f'n_batches = {n_batches}')    
    #-------------------------
    for i, batch_i in enumerate(batch_idxs):
        start = time.time()
        if verbose and (i+1)%n_update==0:
            print(f'{i+1}/{n_batches}')
        i_beg = batch_i[0]
        i_end = batch_i[1]
        #-----
        # NOTE: make_all_columns_lowercase=True because...
        #   EMR would return lowercase outg_rec_nb or outg_rec_nb_gpd_for_sql
        #   Athena maintains the original case, and does not conver to lower case,
        #     so it returns OUTG_REC_NB or OUTG_REC_NB_GPD_FOR_SQL
        end_events_df_i = GenAn.read_df_from_csv_batch(
            paths=paths[i_beg:i_end], 
            cols_and_types_to_convert_dict=cols_and_types_to_convert_dict, 
            to_numeric_errors=to_numeric_errors, 
            make_all_columns_lowercase=True, 
            assert_all_cols_equal=assert_all_cols_equal
        )
        print(f'0: {time.time()-start}')
        #-------------------------
        if end_events_df_i.shape[0]==0:
            continue
        #-------------------------
        start = time.time()
        # Drop any columns from cols_to_drop
        if len(cols_to_drop)>0:
            end_events_df_i = end_events_df_i.drop(columns=cols_to_drop)
        #-----
        # FROM WHAT I CAN TELL, the meters which are 'missing' from meter premise but are present in end events
        #   are caused by meters which were removed or installed close to (a few days/weeks before) an outage.
        # Therefore, although the meter may not have been present at the time of the outage (and therefore was exluced from
        #   meter premise), it could have registered events leading up to/following the outage.
        # e.g., if a meter was removed in the days before an outage, end events are still found for this meter in the days leading up to the outage
        # e.g., if a meter was installed in the days after an outage, end events are still found for this meter in the days following an outage.
        # How should these be handled?
        # The simplest method, which I will implement for now, is to simply ONLY consider those meters which were present
        #   at the time of the outage.  THEREFORE, the two DFs should be joined with an inner merge!
        end_events_df_i = AMIEndEvents.merge_end_events_df_with_mp(
            end_events_df=end_events_df_i, 
            df_mp=mp_df, 
            merge_on_ede=merge_on_ede, 
            merge_on_mp=merge_on_mp, 
            cols_to_include_mp=None, 
            drop_cols = None, 
            rename_cols=None, 
            how='inner', 
            inplace=True
        )
        for grp_by_col in grp_by_cols:
            assert(grp_by_col in end_events_df_i.columns)
        #-------------------------
        dropna_subset_cols = grp_by_cols
        if addtnl_dropna_subset_cols is not None:
            dropna_subset_cols.extend(addtnl_dropna_subset_cols)
        end_events_df_i = end_events_df_i.dropna(subset=dropna_subset_cols)
        #-------------------------
        end_events_df_i=Utilities_dt.strip_tz_info_and_convert_to_dt(
            df=end_events_df_i, 
            time_col='valuesinterval', 
            placement_col='valuesinterval_local', 
            run_quick=True, 
            n_strip=6, 
            inplace=False
        )
        print(f'1: {time.time()-start}')
        start = time.time()
        #---------------------------------------------------------------------------
        # If min_outg_td_window or max_outg_td_window is not None, enforce time restrictions around outages
        if min_outg_td_window is not None or max_outg_td_window is not None:
            #----------
            if not is_no_outage:
                dovs_outgs = DOVSOutages(                 
                    df_construct_type=DFConstructType.kRunSqlQuery, 
                    contstruct_df_args=None, 
                    init_df_in_constructor=True, 
                    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
                    build_sql_function_kwargs=dict(
                        outg_rec_nbs=end_events_df_i[outg_rec_nb_col].unique().tolist(), 
                        from_table_alias='DOV', 
                        datetime_col='DT_OFF_TS_FULL', 
                        cols_of_interest=[
                            'OUTG_REC_NB', 
                            dict(field_desc=f"DOV.DT_ON_TS - DOV.STEP_DRTN_NB/(60*24)", 
                                 alias='DT_OFF_TS_FULL', table_alias_prefix=None)
                        ], 
                        field_to_split='outg_rec_nbs'
                    ),
                )
                outg_dt_off_df = dovs_outgs.df
                outg_dt_off_df = Utilities_df.convert_col_type(df=outg_dt_off_df, column='OUTG_REC_NB', to_type=str)
                outg_dt_off_df=outg_dt_off_df.set_index('OUTG_REC_NB')
                outg_dt_off_series = outg_dt_off_df['DT_OFF_TS_FULL']

                end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
                    end_events_df=end_events_df_i, 
                    outg_times_series=outg_dt_off_series, 
                    min_timedelta=min_outg_td_window, 
                    max_timedelta=max_outg_td_window, 
                    outg_rec_nb_col = outg_rec_nb_col, 
                    datetime_col='valuesinterval_local', 
                    assert_one_time_per_group=True
                )
            else:
                no_outg_time_infos_df = MECPOAn.get_bsln_time_interval_infos_df_from_summary_files(
                    summary_paths=[AMIEndEvents.find_summary_file_from_csv(x) for x in paths[i_beg:i_end]], 
                    output_prem_nbs_col='prem_nbs', 
                    output_t_min_col='t_min', 
                    output_t_max_col='DT_OFF_TS_FULL', 
                    make_addtnl_groupby_idx=True, 
                    include_summary_paths=False
                )
                assert(no_outg_time_infos_df.index.name==trsf_pole_nb_col)
                assert(end_events_df_i[trsf_pole_nb_col].dtype==no_outg_time_infos_df.index.dtype)
                no_outg_time_infos_series = no_outg_time_infos_df['DT_OFF_TS_FULL']    

                end_events_df_i = AMIEndEvents.enforce_end_events_within_interval_of_outage(
                    end_events_df=end_events_df_i, 
                    outg_times_series=no_outg_time_infos_series, 
                    min_timedelta=min_outg_td_window, 
                    max_timedelta=max_outg_td_window, 
                    outg_rec_nb_col = trsf_pole_nb_col, 
                    datetime_col='valuesinterval_local', 
                    assert_one_time_per_group=False
                )
        print(f'2: {time.time()-start}')
        #---------------------------------------------------------------------------
        # After enforcing events within specific time frame, it is possible end_events_df_i is empty.
        # If so, continue
        if end_events_df_i.shape[0]==0:
            continue
        #-------------------------
#         end_events_df_i = AMIEndEvents.extract_reboot_counts_from_reasons_in_df(end_events_df_i)
#         end_events_df_i = AMIEndEvents.extract_fail_reasons_from_reasons_in_df(end_events_df_i)
        #-------------------------
        start = time.time()
        end_events_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(
            df=end_events_df_i, 
        )
        print(f'3: {time.time()-start}')
        #-------------------------
        start = time.time()
        if build_ede_typeid_to_reason_df:
            ede_typeid_to_reason_df_i = AMIEndEvents.build_ede_typeid_to_reason_df(
                end_events_df=end_events_df_i, 
                reason_col='reason', 
                ede_typeid_col='enddeviceeventtypeid'
            )
            if i==0:
                ede_typeid_to_reason_df = ede_typeid_to_reason_df_i.copy()
            else:
                ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
                    ede_typeid_to_reason_df1=ede_typeid_to_reason_df, 
                    ede_typeid_to_reason_df2=ede_typeid_to_reason_df_i,
                    sort=True
                )
        print(f'4: {time.time()-start}')
        #-------------------------
        start = time.time()
        rcpo_i = AMIEndEvents.get_reason_counts_per_group(
            end_events_df = end_events_df_i, 
            group_cols=grp_by_cols, 
            group_freq=None, 
            serial_number_col='serialnumber', 
            reason_col='reason', 
            include_normalize_by_nSNs=include_normalize_by_nSNs, 
            inclue_zero_counts=inclue_zero_counts,
            possible_reasons=None, 
            include_nSNs=True, 
            include_SNs=True, 
            prem_nb_col=prem_nb_col, 
            include_nprem_nbs=include_nprem_nbs,
            include_prem_nbs=include_prem_nbs,   
            return_form = dict(return_multiindex_outg_reason = return_multiindex_outg_reason, 
                               return_normalized_separately  = return_normalized_separately)
        )
        print(f'5: {time.time()-start}')
        #-------------------------
        if rcpo_i.shape[0]==0:
            continue
        #-------------------------
        start = time.time()
        # Include or below in case i=0 rcpo_i comes back empty, causing continue to be called above
        if i==0 or rcpo_full.shape[0]==0:
            rcpo_full = rcpo_i.copy()
        else:
            list_cols=['_SNs']
            list_counts_cols=['_nSNs']
            w_col = '_nSNs'
            if include_prem_nbs:
                list_cols.append('_prem_nbs')
                list_counts_cols.append('_nprem_nbs')
            #-----
            rcpo_full = AMIEndEvents.combine_two_reason_counts_per_outage_dfs(
                rcpo_df_1=rcpo_full, 
                rcpo_df_2=rcpo_i, 
                are_dfs_wide_form=are_dfs_wide_form, 
                normalize_by_nSNs_included=normalize_by_nSNs_included, 
                is_norm=is_norm, 
                list_cols=list_cols, 
                list_counts_cols=list_counts_cols,
                w_col = w_col, 
                level_0_raw_col = 'counts', 
                level_0_nrm_col = 'counts_norm', 
                convert_rcpo_wide_to_long_col_args=None
            )
        print(f'6: {time.time()-start}')
    #-------------------------
    # Drop _gpd_for_sql appendix from any index names 
    rcpo_full = drop_gpd_for_sql_appendix_from_all_index_names(rcpo_full)
    #-------------------------
    if not build_ede_typeid_to_reason_df:
        return rcpo_full
    else:
        return rcpo_full, ede_typeid_to_reason_df

# END NEW